In [ ]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, FancyArrowPatch, Wedge, Rectangle
from matplotlib.collections import PatchCollection
from numpy.typing import NDArray
from typing import Tuple, Optional, Dict, Any, List, Callable

# Local imports
import sys
sys.path.append("../../..")
from src.colors import COLORS

# Configure matplotlib
plt.rcParams.update({
    'figure.figsize': (10, 6),
    'font.size': 11,
    'axes.titlesize': 12,
    'axes.labelsize': 11,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.grid': False,
    'figure.facecolor': 'white',
    'axes.facecolor': 'white'
})

print("Imports successful!")

---
## Section 1: Introduction — What is Hyperscanning?

Traditional neuroscience studies one brain at a time. A participant lies in an fMRI scanner alone, or sits with an EEG cap watching stimuli on a screen. This approach has taught us enormous amounts about how individual brains work — but it misses something crucial about what makes us human.

**Social interaction is a TWO-brain phenomenon.** When you have a conversation, play music together, or collaborate on a task, your brain doesn't operate in isolation — it's constantly adapting to, predicting, and coordinating with another brain. "It takes two to tango," as they say, and yet most of neuroscience has been trying to understand dancing by watching one dancer at a time.

**Hyperscanning** changes this. The term comes from "hyper" (beyond) + "scanning" (brain imaging), meaning: going beyond single-brain recordings to simultaneously capture brain activity from two or more interacting individuals.

### Historical Context

The field began with Montague et al. in 2002, who first demonstrated two-person fMRI while participants played economic games. EEG hyperscanning soon followed, offering practical advantages: better temporal resolution (milliseconds vs seconds), more natural settings, and easier setup for face-to-face interaction.

Today, hyperscanning studies use EEG, fNIRS, fMRI, and even MEG. The core question remains beautifully simple:

> **Do our brains synchronize during social interaction?**
> 
> And if so — what does it mean? What drives it? What are the consequences?

This notebook introduces the foundations of hyperscanning. We'll explore why it matters, what challenges it presents, how to structure the data, and how to think about analyzing inter-brain connectivity. By the end, you'll be ready to dive into the specific connectivity metrics in the following notebooks.

> 💡 **Key insight**: Hyperscanning lets us study the social brain IN social context — capturing the dynamic, bidirectional coupling that makes human interaction so rich.

In [ ]:
# =============================================================================
# Visualization 1: Hyperscanning Concept — Two Brains in Interaction
# =============================================================================

fig, ax = plt.subplots(figsize=(12, 7))

# Draw two stylized heads facing each other
def draw_head(ax: plt.Axes, x_center: float, y_center: float, 
              facing_right: bool, color: str, label: str) -> None:
    """Draw a stylized head with EEG electrodes."""
    # Head circle
    head = Circle((x_center, y_center), 0.35, color=color, alpha=0.3, zorder=1)
    ax.add_patch(head)
    head_outline = Circle((x_center, y_center), 0.35, fill=False, 
                          edgecolor=color, linewidth=2, zorder=2)
    ax.add_patch(head_outline)
    
    # EEG electrodes (small circles on the head)
    electrode_positions = [
        (0.0, 0.30),   # Fz-like
        (0.0, 0.0),    # Cz-like
        (0.0, -0.25),  # Pz-like
        (-0.20, 0.15), # F3/F4-like
        (0.20, 0.15),
        (-0.20, -0.10), # C3/C4-like
        (0.20, -0.10),
    ]
    for dx, dy in electrode_positions:
        electrode = Circle((x_center + dx, y_center + dy), 0.04, 
                          color=color, zorder=3)
        ax.add_patch(electrode)
    
    # Nose indicator (triangle pointing in facing direction)
    nose_x = x_center + (0.35 if facing_right else -0.35)
    nose_dx = 0.1 if facing_right else -0.1
    ax.plot([nose_x, nose_x + nose_dx, nose_x], 
            [y_center + 0.05, y_center, y_center - 0.05],
            color=color, linewidth=2, zorder=2)
    
    # Label
    ax.text(x_center, y_center - 0.55, label, ha='center', va='top',
            fontsize=14, fontweight='bold', color=color)

# Draw two heads
draw_head(ax, -0.7, 0, facing_right=True, color=COLORS["signal_1"], label="Participant 1")
draw_head(ax, 0.7, 0, facing_right=False, color=COLORS["signal_2"], label="Participant 2")

# Draw brain waves emanating from each head
t = np.linspace(0, 2 * np.pi, 100)

# Waves from P1 (going right)
for i, y_offset in enumerate([-0.15, 0, 0.15]):
    wave_x = np.linspace(-0.3, 0.0, 50)
    wave_y = 0.05 * np.sin(6 * wave_x * np.pi + i) + y_offset
    alpha = 0.8 - i * 0.2
    ax.plot(wave_x, wave_y, color=COLORS["signal_1"], alpha=alpha, linewidth=2)

# Waves from P2 (going left)
for i, y_offset in enumerate([-0.15, 0, 0.15]):
    wave_x = np.linspace(0.0, 0.3, 50)
    wave_y = 0.05 * np.sin(6 * wave_x * np.pi + i + 0.3) + y_offset
    alpha = 0.8 - i * 0.2
    ax.plot(wave_x, wave_y, color=COLORS["signal_2"], alpha=alpha, linewidth=2)

# Central synchronization symbol
sync_circle = Circle((0, 0), 0.12, fill=False, edgecolor=COLORS["high_sync"],
                      linewidth=3, linestyle='--', zorder=4)
ax.add_patch(sync_circle)
ax.text(0, 0, "⟷", ha='center', va='center', fontsize=20, 
        color=COLORS["high_sync"], fontweight='bold', zorder=5)

# Title and annotations
ax.text(0, 0.7, "Hyperscanning", ha='center', va='bottom',
        fontsize=18, fontweight='bold', color=COLORS["text"])
ax.text(0, 0.55, "Simultaneous brain recording from interacting individuals",
        ha='center', va='bottom', fontsize=11, style='italic', color=COLORS["grid"])

# Bottom annotation
ax.text(0, -0.75, "Do their brain signals synchronize?",
        ha='center', va='top', fontsize=12, color=COLORS["text"])

ax.set_xlim(-1.5, 1.5)
ax.set_ylim(-0.9, 0.9)
ax.set_aspect('equal')
ax.axis('off')

plt.tight_layout()
plt.show()

print("✓ Visualization 1: Hyperscanning concept")
print("  Two participants with EEG caps, brain signals potentially synchronizing")
print("  The central question: do brains couple during social interaction?")

---
## Section 2: Why Hyperscanning Matters

### The Social Brain Hypothesis

Human brains are remarkable for many reasons, but perhaps most remarkable is our social cognition. The "social brain hypothesis" proposes that our large brains evolved primarily to handle the complexities of social life — keeping track of relationships, predicting others' behavior, cooperating and competing in groups.

Yet traditional neuroscience has mostly studied social cognition in isolation:
- Watch videos of faces → not a real interaction
- Imagine social scenarios → artificial, no actual partner
- Play games against a computer → no real stakes of social reputation

What's missing is the **real-time, bidirectional, dynamic exchange** that characterizes actual human interaction.

### What Hyperscanning Reveals

Studies using hyperscanning have discovered fascinating phenomena:

- **Inter-brain synchrony correlates with rapport**: People who "click" show more synchronized brain activity
- **Synchrony predicts cooperation success**: Teams with higher brain coupling perform better
- **Leader-follower dynamics are visible**: One brain can "lead" another in time
- **Shared attention creates shared patterns**: Looking at the same thing together synchronizes brains

### Applications Across Domains

Hyperscanning has applications far beyond basic research:

| Domain | Application | Finding |
|--------|-------------|----------|
| **Education** | Teacher-student dynamics | Synchrony predicts learning outcomes |
| **Clinical** | Therapist-patient rapport | Synchrony relates to therapeutic alliance |
| **Development** | Parent-child bonding | Attachment quality visible in brain coupling |
| **Performance** | Music, sports, teams | Coordination linked to neural synchrony |
| **Communication** | Conversation, storytelling | Speaker-listener coupling during understanding |

In [ ]:
# =============================================================================
# Visualization 2: Applications of Hyperscanning
# =============================================================================

fig, ax = plt.subplots(figsize=(10, 10))

# Application domains with text abbreviations (no emojis for font compatibility)
domains = [
    {"name": "Education", "abbrev": "EDU", "desc": "Teacher-student\nlearning"},
    {"name": "Clinical", "abbrev": "CLN", "desc": "Therapist-patient\nrapport"},
    {"name": "Development", "abbrev": "DEV", "desc": "Parent-child\nbonding"},
    {"name": "Music", "abbrev": "MUS", "desc": "Musical\ncoordination"},
    {"name": "Sports", "abbrev": "SPT", "desc": "Team\ncoordination"},
    {"name": "Communication", "abbrev": "COM", "desc": "Conversation\nunderstanding"},
]

# Define colors for each domain
domain_colors = [
    COLORS["signal_1"],  # Education
    COLORS["signal_2"],  # Clinical
    COLORS["signal_3"],  # Development
    COLORS["signal_4"],  # Music
    COLORS["signal_5"],  # Sports
    COLORS["signal_6"],  # Communication
]

# Draw central hub
center_circle = Circle((0, 0), 0.25, color=COLORS["high_sync"], alpha=0.9, zorder=3)
ax.add_patch(center_circle)
ax.text(0, 0.03, "Hyper-", ha='center', va='center', fontsize=12, 
        fontweight='bold', color='white', zorder=4)
ax.text(0, -0.08, "scanning", ha='center', va='center', fontsize=12, 
        fontweight='bold', color='white', zorder=4)

# Draw domains around center
n_domains = len(domains)
radius = 0.7
angles = np.linspace(np.pi/2, np.pi/2 - 2*np.pi, n_domains, endpoint=False)

for i, (domain, angle, color) in enumerate(zip(domains, angles, domain_colors)):
    x = radius * np.cos(angle)
    y = radius * np.sin(angle)
    
    # Domain circle (slightly more opaque)
    domain_circle = Circle((x, y), 0.18, color=color, alpha=0.4, zorder=2)
    ax.add_patch(domain_circle)
    domain_outline = Circle((x, y), 0.18, fill=False, edgecolor=color, 
                            linewidth=2.5, zorder=2)
    ax.add_patch(domain_outline)
    
    # Connection line to center
    ax.plot([0, x * 0.5], [0, y * 0.5], color=color, linewidth=2, 
            alpha=0.6, zorder=1)
    
    # Abbreviation text inside circle (instead of emoji)
    ax.text(x, y, domain["abbrev"], ha='center', va='center', 
            fontsize=11, fontweight='bold', color=color, zorder=3)
    
    # Domain name (outside)
    text_radius = radius + 0.28
    text_x = text_radius * np.cos(angle)
    text_y = text_radius * np.sin(angle)
    ax.text(text_x, text_y, domain["name"], ha='center', va='center',
            fontsize=13, fontweight='bold', color=color)
    
    # Description (even further out) - use darker color for readability
    desc_radius = radius + 0.45
    desc_x = desc_radius * np.cos(angle)
    desc_y = desc_radius * np.sin(angle)
    ax.text(desc_x, desc_y, domain["desc"], ha='center', va='center',
            fontsize=10, color=COLORS["text"], style='italic')

ax.set_xlim(-1.4, 1.4)
ax.set_ylim(-1.4, 1.4)
ax.set_aspect('equal')
ax.axis('off')

ax.set_title("Hyperscanning Applications Across Domains", 
             fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
plt.show()

print("✓ Visualization 2: Hyperscanning applications")
print("  Research spans education, clinical, developmental, and performance domains")
print("  Common thread: understanding brain coupling during social interaction")

---
## Section 3: Common Hyperscanning Paradigms

Hyperscanning studies use diverse experimental paradigms, each designed to probe different aspects of social interaction. Here are the main categories:

### Cooperation Tasks
Participants work together toward a shared goal:
- **Joint button pressing**: Synchronize actions to hit targets together
- **Cooperative games**: Solve puzzles or navigate challenges as a team
- **Construction tasks**: Build something together (physical or virtual)

*Research question*: Does neural synchrony support successful cooperation?

### Communication Tasks
Information exchange between participants:
- **Conversation**: Structured or free-form dialogue
- **Storytelling**: One speaks, one listens
- **Teaching-learning**: Knowledge transfer situations

*Research question*: How does information flow between brains during communication?

### Joint Attention Tasks
Sharing focus on the same object or event:
- **Shared visual attention**: Looking at the same thing together
- **Following gaze**: One directs, one follows
- **Object manipulation**: Jointly attending to manipulated objects

*Research question*: Does shared attention synchronize brains?

### Imitation & Synchronization Tasks
Coordinating movements:
- **Mirror movements**: Copy each other's gestures
- **Musical coordination**: Drumming, singing, playing together
- **Dance**: Coordinated movement to music

*Research question*: Does behavioral synchrony drive neural synchrony?

### Competition Tasks
Opposing goals:
- **Economic games**: Prisoner's dilemma, ultimatum game
- **Strategic games**: Chess, competitive video games
- **Sports competition**: Head-to-head physical competition

*Research question*: Does competition synchronize or desynchronize brains?

### Naturalistic Paradigms
Unconstrained interaction:
- **Free conversation**: Natural dialogue without script
- **Real-world settings**: Cafes, classrooms, living rooms

*Research question*: What happens in ecologically valid contexts?

In [ ]:
# =============================================================================
# Visualization 3: Hyperscanning Paradigms
# =============================================================================

fig, axes = plt.subplots(2, 3, figsize=(14, 9))

# Define paradigms with simple visual representations
paradigms = [
    {"name": "Cooperation", "desc": "Working together\ntoward shared goal"},
    {"name": "Communication", "desc": "Speaking and\nlistening"},
    {"name": "Joint Attention", "desc": "Looking at the\nsame thing"},
    {"name": "Imitation", "desc": "Coordinating\nmovements"},
    {"name": "Competition", "desc": "Opposing\ngoals"},
    {"name": "Naturalistic", "desc": "Free\ninteraction"},
]

paradigm_colors = [
    COLORS["signal_1"], COLORS["signal_2"], COLORS["signal_3"],
    COLORS["signal_4"], COLORS["signal_5"], COLORS["signal_6"],
]

for idx, (ax, paradigm, color) in enumerate(zip(axes.flat, paradigms, paradigm_colors)):
    # Draw two simple heads
    head_y = 0.3
    
    # Left head
    left_head = Circle((-0.4, head_y), 0.25, color=COLORS["signal_1"], alpha=0.6)
    ax.add_patch(left_head)
    
    # Right head
    right_head = Circle((0.4, head_y), 0.25, color=COLORS["signal_2"], alpha=0.6)
    ax.add_patch(right_head)
    
    # Task-specific visual element
    if paradigm["name"] == "Cooperation":
        # Arrows pointing to same target
        ax.annotate("", xy=(0, -0.1), xytext=(-0.3, 0.1),
                   arrowprops=dict(arrowstyle="->", color=color, lw=2))
        ax.annotate("", xy=(0, -0.1), xytext=(0.3, 0.1),
                   arrowprops=dict(arrowstyle="->", color=color, lw=2))
        target = Circle((0, -0.2), 0.1, color=color, alpha=0.8)
        ax.add_patch(target)
        
    elif paradigm["name"] == "Communication":
        # Speech waves
        for i, offset in enumerate([0.1, 0.2, 0.3]):
            wave_x = np.linspace(-0.1 + offset, 0.1 + offset, 20)
            wave_y = 0.05 * np.sin(wave_x * 30) + head_y
            ax.plot(wave_x, wave_y, color=color, lw=2, alpha=0.8-i*0.2)
            
    elif paradigm["name"] == "Joint Attention":
        # Both looking at same object
        obj = Circle((0, -0.15), 0.12, color=color, alpha=0.8)
        ax.add_patch(obj)
        ax.plot([-0.2, 0], [0.15, -0.05], color=COLORS["signal_1"], lw=2, ls='--', alpha=0.6)
        ax.plot([0.2, 0], [0.15, -0.05], color=COLORS["signal_2"], lw=2, ls='--', alpha=0.6)
        
    elif paradigm["name"] == "Imitation":
        # Mirrored arrows
        ax.annotate("", xy=(-0.2, -0.1), xytext=(-0.5, -0.1),
                   arrowprops=dict(arrowstyle="->", color=color, lw=2))
        ax.annotate("", xy=(0.5, -0.1), xytext=(0.2, -0.1),
                   arrowprops=dict(arrowstyle="->", color=color, lw=2))
        ax.text(0, -0.1, "↔", ha='center', va='center', fontsize=20, color=color)
        
    elif paradigm["name"] == "Competition":
        # Opposing arrows
        ax.annotate("", xy=(0.05, -0.05), xytext=(-0.3, -0.05),
                   arrowprops=dict(arrowstyle="->", color=COLORS["signal_1"], lw=2))
        ax.annotate("", xy=(-0.05, -0.15), xytext=(0.3, -0.15),
                   arrowprops=dict(arrowstyle="->", color=COLORS["signal_2"], lw=2))
        ax.text(0, -0.1, "⚔", ha='center', va='center', fontsize=18, color=color)
        
    elif paradigm["name"] == "Naturalistic":
        # Free wavy lines between them
        x_wave = np.linspace(-0.15, 0.15, 40)
        for i in range(3):
            y_wave = 0.08 * np.sin(x_wave * 15 + i) + head_y - 0.35 - i * 0.1
            ax.plot(x_wave, y_wave, color=color, lw=2, alpha=0.6)
    
    # Title
    ax.set_title(paradigm["name"], fontsize=13, fontweight='bold', color=color, pad=10)
    
    # Description below
    ax.text(0, -0.5, paradigm["desc"], ha='center', va='top', fontsize=9,
            color=COLORS["grid"], style='italic')
    
    ax.set_xlim(-0.8, 0.8)
    ax.set_ylim(-0.65, 0.7)
    ax.set_aspect('equal')
    ax.axis('off')

plt.suptitle("Common Hyperscanning Paradigms", fontsize=15, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("✓ Visualization 3: Six categories of hyperscanning paradigms")
print("  Each paradigm probes different aspects of social interaction")
print("  Choice depends on research question and practical constraints")

---
## Section 4: Inter-Brain Synchrony — The Core Concept

**Inter-brain synchrony (IBS)** is the central phenomenon of interest in hyperscanning. It refers to the statistical coupling or correlation between brain signals from two different individuals.

### What We Mean by Synchrony

When we say two brains are "synchronized," we mean their signals show some form of statistical dependency:

- **Phase synchrony**: The phases of neural oscillations align across brains
- **Amplitude coupling**: Power fluctuations rise and fall together
- **Frequency coupling**: Activity in specific bands correlates
- **Directed coupling**: One brain's activity predicts the other's

### Important Clarification

Inter-brain synchrony is **NOT**:
- ❌ Telepathy or direct brain-to-brain communication
- ❌ Identical brain activity
- ❌ Proof that brains are "connected"

It **IS**:
- ✅ Statistical dependency between two brain signals
- ✅ Evidence of shared processing or coordinated dynamics
- ✅ A correlate of social interaction quality

### Connection to What We've Learned

Here's the beautiful insight: **everything we learned in the foundations applies!**

| Foundation Topic | Application in Hyperscanning |
|------------------|------------------------------|
| Phase (B02) | Inter-brain phase synchrony |
| Amplitude (B03) | Inter-brain amplitude correlation |
| Coherence (coming in F01) | Inter-brain coherence |
| PLV (coming in G01) | Inter-brain PLV |
| Transfer Entropy (D03) | Inter-brain directed influence |

The metrics are identical — we just apply them BETWEEN brains instead of between channels within one brain.

In [ ]:
# =============================================================================
# Visualization 4: Inter-Brain Synchrony Example
# =============================================================================

np.random.seed(42)

# Create simulated EEG from two participants with CLEAR periods of sync/desync
fs = 256  # Hz
duration = 6  # seconds (longer for better illustration)
t = np.arange(0, duration, 1/fs)
n_samples = len(t)

# Define synchrony periods explicitly
# Format: (start_time, end_time, is_synchronized)
sync_periods = [
    (0.0, 1.0, False),   # Desync
    (1.0, 2.2, True),    # HIGH SYNC
    (2.2, 3.3, False),   # Desync
    (3.3, 4.8, True),    # HIGH SYNC
    (4.8, 6.0, False),   # Desync
]

# Base alpha frequency
freq = 10  # Hz

# Generate signals with controlled synchrony
signal_p1 = np.zeros(n_samples)
signal_p2 = np.zeros(n_samples)

for start, end, is_sync in sync_periods:
    start_idx = int(start * fs)
    end_idx = int(end * fs)
    segment_t = t[start_idx:end_idx]
    segment_len = end_idx - start_idx
    
    # Participant 1: always has consistent alpha
    phase_1 = 2 * np.pi * freq * segment_t
    seg_p1 = np.sin(phase_1) + 0.3 * np.random.randn(segment_len)
    
    if is_sync:
        # SYNCHRONIZED: P2 follows P1 closely (small phase difference)
        phase_2 = phase_1 + 0.2  # Small constant phase lag
        seg_p2 = np.sin(phase_2) + 0.3 * np.random.randn(segment_len)
    else:
        # DESYNCHRONIZED: P2 has different/drifting phase
        phase_drift = np.cumsum(np.random.randn(segment_len) * 0.15)
        phase_2 = 2 * np.pi * (freq + 0.5) * segment_t + phase_drift
        seg_p2 = np.sin(phase_2) + 0.4 * np.random.randn(segment_len)
    
    signal_p1[start_idx:end_idx] = seg_p1
    signal_p2[start_idx:end_idx] = seg_p2

# Smooth transitions at boundaries
from scipy.ndimage import gaussian_filter1d
signal_p1 = gaussian_filter1d(signal_p1, sigma=3)
signal_p2 = gaussian_filter1d(signal_p2, sigma=3)

# Create figure
fig, axes = plt.subplots(3, 1, figsize=(14, 8), height_ratios=[1, 1, 0.8])

# Plot participant 1
ax1 = axes[0]
ax1.plot(t, signal_p1, color=COLORS["signal_1"], linewidth=0.8)
ax1.set_ylabel("Amplitude (µV)", fontsize=11)
ax1.set_title("Participant 1 — Frontal electrode (Fz)", fontsize=12, fontweight='bold',
              color=COLORS["signal_1"])
ax1.set_xlim(0, duration)
ax1.set_ylim(-2, 2)
ax1.axhline(0, color=COLORS["grid"], linewidth=0.5, alpha=0.5)

# Plot participant 2
ax2 = axes[1]
ax2.plot(t, signal_p2, color=COLORS["signal_2"], linewidth=0.8)
ax2.set_ylabel("Amplitude (µV)", fontsize=11)
ax2.set_title("Participant 2 — Frontal electrode (Fz)", fontsize=12, fontweight='bold',
              color=COLORS["signal_2"])
ax2.set_xlim(0, duration)
ax2.set_ylim(-2, 2)
ax2.axhline(0, color=COLORS["grid"], linewidth=0.5, alpha=0.5)

# Bottom panel: compute ACTUAL synchrony using sliding window correlation
ax3 = axes[2]

window_size = int(0.4 * fs)  # 400 ms window
step = int(0.05 * fs)  # 50 ms step (smoother)
n_windows = (n_samples - window_size) // step

sync_values = []
sync_times = []

for i in range(n_windows):
    start_idx = i * step
    end_idx = start_idx + window_size
    
    # Correlation as synchrony measure
    corr = np.corrcoef(signal_p1[start_idx:end_idx], signal_p2[start_idx:end_idx])[0, 1]
    sync_values.append(max(0, corr))  # Keep positive values
    sync_times.append(t[start_idx + window_size // 2])

sync_times = np.array(sync_times)
sync_values = np.array(sync_values)

# Now highlight regions based on ACTUAL computed synchrony
threshold = 0.5
high_sync_mask = sync_values > threshold

# Find contiguous high-sync regions for highlighting
in_high_sync = False
highlight_regions = []
for i, (time, is_high) in enumerate(zip(sync_times, high_sync_mask)):
    if is_high and not in_high_sync:
        region_start = time
        in_high_sync = True
    elif not is_high and in_high_sync:
        region_end = sync_times[i-1]
        highlight_regions.append((region_start, region_end))
        in_high_sync = False
if in_high_sync:
    highlight_regions.append((region_start, sync_times[-1]))

# Apply highlights to signal plots
for start, end in highlight_regions:
    ax1.axvspan(start, end, alpha=0.25, color=COLORS["high_sync"], zorder=0)
    ax2.axvspan(start, end, alpha=0.25, color=COLORS["high_sync"], zorder=0)

# Add "High sync" labels only on significant regions (> 0.5s duration)
significant_regions = [(s, e) for s, e in highlight_regions if (e - s) > 0.5]
for i, (start, end) in enumerate(significant_regions[:2]):  # Label first two
    mid = (start + end) / 2
    ax1.text(mid, 1.7, "High sync", fontsize=10, color=COLORS["high_sync"], 
             fontweight='bold', ha='center')

# Plot synchrony curve
ax3.fill_between(sync_times, 0, sync_values, alpha=0.3, color=COLORS["high_sync"])
ax3.plot(sync_times, sync_values, color=COLORS["high_sync"], linewidth=2)
ax3.axhline(threshold, color=COLORS["grid"], linestyle='--', linewidth=1.5, alpha=0.8)
ax3.text(duration - 0.1, threshold + 0.05, "Threshold", fontsize=9, 
         color=COLORS["grid"], ha='right')
ax3.set_xlabel("Time (s)", fontsize=11)
ax3.set_ylabel("Synchrony", fontsize=11)
ax3.set_title("Inter-Brain Synchrony (sliding window correlation)", fontsize=12, fontweight='bold')
ax3.set_xlim(0, duration)
ax3.set_ylim(0, 1)

# Shade high sync regions in bottom panel too
for start, end in highlight_regions:
    ax3.axvspan(start, end, alpha=0.15, color=COLORS["high_sync"], zorder=0)

plt.tight_layout()
plt.show()

print("✓ Visualization 4: Inter-brain synchrony example")
print("  Signals are TRULY synchronized in purple regions (in-phase oscillations)")
print("  Signals are desynchronized in white regions (different phases)")
print(f"  Synchrony ranges from {sync_values.min():.2f} to {sync_values.max():.2f}")
print(f"  High sync periods: {len(highlight_regions)} detected above threshold {threshold}")

---
## Section 5: Unique Challenges of Hyperscanning

Hyperscanning brings unique methodological challenges that don't exist in single-brain studies. Being aware of these challenges is crucial for designing and interpreting studies correctly.

### Challenge 1: No Volume Conduction Between Brains ✅

Wait — this is actually an **ADVANTAGE**! 

Remember from notebook C01 how volume conduction creates spurious connectivity within a single brain? Different electrodes pick up the same source, creating fake correlations.

In hyperscanning, the two brains are in **separate heads**. There's no shared skull or tissue to conduct signals between them. So any synchrony we observe cannot be an artifact of volume conduction!

However, we must still worry about:
- Shared electromagnetic interference (e.g., power line noise)
- Movement artifacts if participants move together
- Common physiological rhythms (breathing, heartbeat)

### Challenge 2: Behavioral Confounds

People naturally synchronize their behavior during interaction:
- They match each other's speech patterns
- They mirror movements
- They breathe in sync

This behavioral synchrony can create neural synchrony that's not truly "social":
- Synchronized movements → synchronized motor artifacts (EMG)
- Same visual input → similar visual processing
- Matched breathing → similar slow oscillations

**Question**: Is the neural synchrony we observe something BEYOND behavioral synchrony?

### Challenge 3: Stimulus-Driven vs. Interaction-Driven Synchrony

This is perhaps the most important confound to understand. If both participants:
- See the same screen
- Hear the same sounds
- Experience the same experimental events

...then both brains will respond similarly to these shared stimuli. This creates "synchrony" that has nothing to do with social interaction — it's just both brains processing the same input!

**Solution**: Pseudo-pair analysis (we'll cover this in detail later).

### Challenge 4: Data Structure Complexity

Single-brain connectivity gives us an $n \times n$ matrix for $n$ channels.

Two-brain connectivity gives us:
- Within-P1: $n_1 \times n_1$
- Within-P2: $n_2 \times n_2$  
- Between: $n_1 \times n_2$

Or combined: $(n_1 + n_2) \times (n_1 + n_2)$

More channels, more pairs, more multiple comparisons!

In [ ]:
# =============================================================================
# Visualization 5: The Stimulus-Driven Confound
# =============================================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Left panel: Stimulus-driven synchrony (problematic)
ax1 = axes[0]

# Draw screen in center
screen = Rectangle((-0.15, 0.4), 0.3, 0.25, color=COLORS["grid"], alpha=0.5)
ax1.add_patch(screen)
ax1.text(0, 0.55, "Shared\nStimulus", ha='center', va='center', fontsize=10, 
         fontweight='bold', color='white')

# Two heads looking at screen
head1 = Circle((-0.5, 0), 0.2, color=COLORS["signal_1"], alpha=0.6)
ax1.add_patch(head1)
ax1.text(-0.5, -0.35, "P1", ha='center', fontsize=11, fontweight='bold', 
         color=COLORS["signal_1"])

head2 = Circle((0.5, 0), 0.2, color=COLORS["signal_2"], alpha=0.6)
ax1.add_patch(head2)
ax1.text(0.5, -0.35, "P2", ha='center', fontsize=11, fontweight='bold',
         color=COLORS["signal_2"])

# Arrows from screen to both heads (stimulus input)
ax1.annotate("", xy=(-0.35, 0.15), xytext=(-0.1, 0.42),
            arrowprops=dict(arrowstyle="->", color=COLORS["grid"], lw=2))
ax1.annotate("", xy=(0.35, 0.15), xytext=(0.1, 0.42),
            arrowprops=dict(arrowstyle="->", color=COLORS["grid"], lw=2))

# "Synchrony" between brains (but it's spurious!)
ax1.annotate("", xy=(0.25, 0), xytext=(-0.25, 0),
            arrowprops=dict(arrowstyle="<->", color=COLORS["low_sync"], 
                           lw=3, ls='--'))
ax1.text(0, -0.08, "Apparent\nsynchrony", ha='center', va='top', fontsize=9,
        color=COLORS["low_sync"], style='italic')

# Warning sign
ax1.text(0, -0.65, "⚠ Both brains respond to same stimulus", ha='center',
        fontsize=11, color=COLORS["negative"], fontweight='bold')
ax1.text(0, -0.8, "Not social synchrony — just shared input!", ha='center',
        fontsize=10, color=COLORS["grid"], style='italic')

ax1.set_xlim(-0.9, 0.9)
ax1.set_ylim(-0.9, 0.8)
ax1.set_aspect('equal')
ax1.axis('off')
ax1.set_title("Stimulus-Driven Synchrony (Confound)", fontsize=13, 
              fontweight='bold', color=COLORS["negative"])

# Right panel: True interaction-driven synchrony
ax2 = axes[1]

# Two heads facing each other (no shared screen)
head1 = Circle((-0.4, 0), 0.2, color=COLORS["signal_1"], alpha=0.6)
ax2.add_patch(head1)
ax2.text(-0.4, -0.35, "P1", ha='center', fontsize=11, fontweight='bold',
         color=COLORS["signal_1"])

head2 = Circle((0.4, 0), 0.2, color=COLORS["signal_2"], alpha=0.6)
ax2.add_patch(head2)
ax2.text(0.4, -0.35, "P2", ha='center', fontsize=11, fontweight='bold',
         color=COLORS["signal_2"])

# Bidirectional arrows between heads
ax2.annotate("", xy=(0.15, 0.05), xytext=(-0.15, 0.05),
            arrowprops=dict(arrowstyle="->", color=COLORS["high_sync"], lw=2))
ax2.annotate("", xy=(-0.15, -0.05), xytext=(0.15, -0.05),
            arrowprops=dict(arrowstyle="->", color=COLORS["high_sync"], lw=2))

# Synchrony symbol
sync_circle = Circle((0, 0), 0.08, fill=False, edgecolor=COLORS["high_sync"],
                     linewidth=2, linestyle='-')
ax2.add_patch(sync_circle)

# Speech bubbles / interaction symbols
ax2.text(-0.15, 0.25, "...", fontsize=14, color=COLORS["signal_1"], ha='center')
ax2.text(0.15, 0.25, "...", fontsize=14, color=COLORS["signal_2"], ha='center')

# Positive message
ax2.text(0, -0.65, "✓ Synchrony from actual interaction", ha='center',
        fontsize=11, color=COLORS["positive"], fontweight='bold')
ax2.text(0, -0.8, "Bidirectional coupling through communication", ha='center',
        fontsize=10, color=COLORS["grid"], style='italic')

ax2.set_xlim(-0.9, 0.9)
ax2.set_ylim(-0.9, 0.8)
ax2.set_aspect('equal')
ax2.axis('off')
ax2.set_title("Interaction-Driven Synchrony (Real)", fontsize=13,
              fontweight='bold', color=COLORS["positive"])

plt.tight_layout()
plt.show()

print("✓ Visualization 5: Stimulus-driven vs interaction-driven synchrony")
print("  Left: Confound where both participants respond to same stimulus")
print("  Right: True social synchrony from bidirectional interaction")

---
## Section 6: Data Structure for Hyperscanning

Understanding how to organize hyperscanning data is essential before any analysis. Unlike single-brain studies, we now have **two recordings** that must be aligned and combined properly.

### Recording Setup

In practice, hyperscanning can be done with:
- **Two separate EEG systems** — requires careful synchronization via triggers
- **One system with dual cap** — easier synchronization but limited channel count
- **Multiple modalities** — EEG + fNIRS, dual-fMRI, etc.

The critical requirement: **temporal synchronization**. Both recordings must be aligned to the same time base, typically using shared trigger signals.

### Data Dimensions

For each participant, we have:
- **Participant 1**: shape `(n_channels_P1, n_samples)`
- **Participant 2**: shape `(n_channels_P2, n_samples)`

Often `n_channels_P1 = n_channels_P2`, but not always (e.g., if one cap has a bad channel).

### Connectivity Blocks

When computing connectivity for hyperscanning, we get THREE distinct blocks:

| Block | Description | Shape | Meaning |
|-------|-------------|-------|---------|
| **Within-P1** | Connectivity among P1's channels | `(n_P1, n_P1)` | Single-brain connectivity for P1 |
| **Within-P2** | Connectivity among P2's channels | `(n_P2, n_P2)` | Single-brain connectivity for P2 |
| **Between** | Connectivity from P1 to P2 | `(n_P1, n_P2)` | **Inter-brain connectivity** |

The "Between" block is what makes hyperscanning unique — it captures coupling **across** brains.

### Combined Matrix Structure

We can combine these blocks into a full connectivity matrix:

```
                P1 channels    P2 channels
P1 channels [   Within-P1   |   Between    ]
P2 channels [   Between.T   |   Within-P2  ]
```

This $(n_{P1} + n_{P2}) \times (n_{P1} + n_{P2})$ matrix contains all connectivity information.

### Channel Naming Convention

To avoid confusion, we prefix channel names:
- P1 channels: `P1_Fz`, `P1_Cz`, `P1_Pz`, ...
- P2 channels: `P2_Fz`, `P2_Cz`, `P2_Pz`, ...

This makes it immediately clear which participant each channel belongs to.

In [ ]:
# =============================================================================
# Visualization 6: Hyperscanning Data Structure
# =============================================================================

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Panel A: Two separate data arrays
ax1 = axes[0]
ax1.set_title("A. Raw Data Arrays", fontsize=12, fontweight='bold', pad=10)

# P1 data array
p1_rect = Rectangle((0.1, 0.55), 0.35, 0.35, color=COLORS["signal_1"], alpha=0.6)
ax1.add_patch(p1_rect)
ax1.text(0.275, 0.725, "P1 Data\n(n_ch₁ × n_samples)", ha='center', va='center', 
         fontsize=10, fontweight='bold', color='white')

# P2 data array
p2_rect = Rectangle((0.55, 0.55), 0.35, 0.35, color=COLORS["signal_2"], alpha=0.6)
ax1.add_patch(p2_rect)
ax1.text(0.725, 0.725, "P2 Data\n(n_ch₂ × n_samples)", ha='center', va='center', 
         fontsize=10, fontweight='bold', color='white')

# Arrow down
ax1.annotate("", xy=(0.5, 0.35), xytext=(0.5, 0.5),
            arrowprops=dict(arrowstyle="->", color=COLORS["text"], lw=2))
ax1.text(0.5, 0.42, "Combine", ha='center', va='center', fontsize=9, color=COLORS["text"])

# Combined array
combined_rect = Rectangle((0.15, 0.08), 0.7, 0.22, color=COLORS["high_sync"], alpha=0.4)
ax1.add_patch(combined_rect)
# Show P1 and P2 sections
ax1.axvline(x=0.5, ymin=0.08/1.0, ymax=0.3/1.0, color=COLORS["grid"], linestyle='--', lw=1)
ax1.text(0.325, 0.19, "P1", ha='center', va='center', fontsize=11, 
         fontweight='bold', color=COLORS["signal_1"])
ax1.text(0.675, 0.19, "P2", ha='center', va='center', fontsize=11, 
         fontweight='bold', color=COLORS["signal_2"])
ax1.text(0.5, 0.02, "Combined: (n_ch₁ + n_ch₂) × n_samples", ha='center', 
         fontsize=9, color=COLORS["text"])

ax1.set_xlim(0, 1)
ax1.set_ylim(0, 1)
ax1.axis('off')

# Panel B: Connectivity matrix blocks
ax2 = axes[1]
ax2.set_title("B. Connectivity Matrix Blocks", fontsize=12, fontweight='bold', pad=10)

# Draw the 2x2 block structure
block_size = 0.35
gap = 0.05
start_x = 0.15
start_y = 0.15

# Within-P1 (top-left)
within_p1 = Rectangle((start_x, start_y + block_size + gap), block_size, block_size, 
                       color=COLORS["signal_1"], alpha=0.5)
ax2.add_patch(within_p1)
ax2.text(start_x + block_size/2, start_y + block_size + gap + block_size/2, 
         "Within\nP1", ha='center', va='center', fontsize=10, fontweight='bold')

# Between (top-right)
between_rect = Rectangle((start_x + block_size + gap, start_y + block_size + gap), 
                          block_size, block_size, color=COLORS["high_sync"], alpha=0.6)
ax2.add_patch(between_rect)
ax2.text(start_x + block_size + gap + block_size/2, start_y + block_size + gap + block_size/2, 
         "Between\nP1↔P2", ha='center', va='center', fontsize=10, fontweight='bold', color='white')

# Between.T (bottom-left)
between_t_rect = Rectangle((start_x, start_y), block_size, block_size, 
                            color=COLORS["high_sync"], alpha=0.6)
ax2.add_patch(between_t_rect)
ax2.text(start_x + block_size/2, start_y + block_size/2, 
         "Between\n(transpose)", ha='center', va='center', fontsize=10, fontweight='bold', color='white')

# Within-P2 (bottom-right)
within_p2 = Rectangle((start_x + block_size + gap, start_y), block_size, block_size, 
                       color=COLORS["signal_2"], alpha=0.5)
ax2.add_patch(within_p2)
ax2.text(start_x + block_size + gap + block_size/2, start_y + block_size/2, 
         "Within\nP2", ha='center', va='center', fontsize=10, fontweight='bold')

# Labels
ax2.text(start_x + block_size/2, start_y + 2*block_size + gap + 0.08, "P1 ch", 
         ha='center', fontsize=10, color=COLORS["signal_1"], fontweight='bold')
ax2.text(start_x + block_size + gap + block_size/2, start_y + 2*block_size + gap + 0.08, "P2 ch", 
         ha='center', fontsize=10, color=COLORS["signal_2"], fontweight='bold')
ax2.text(start_x - 0.08, start_y + block_size + gap + block_size/2, "P1", 
         ha='center', va='center', fontsize=10, color=COLORS["signal_1"], fontweight='bold', rotation=90)
ax2.text(start_x - 0.08, start_y + block_size/2, "P2", 
         ha='center', va='center', fontsize=10, color=COLORS["signal_2"], fontweight='bold', rotation=90)

ax2.set_xlim(0, 1)
ax2.set_ylim(0, 1)
ax2.axis('off')

# Panel C: Example with real channel names
ax3 = axes[2]
ax3.set_title("C. Channel Naming Example", fontsize=12, fontweight='bold', pad=10)

# Create a small example matrix
channels_p1 = ["P1_Fz", "P1_Cz", "P1_Pz"]
channels_p2 = ["P2_Fz", "P2_Cz", "P2_Pz"]
all_channels = channels_p1 + channels_p2

# Create simulated connectivity values
np.random.seed(123)
n_total = 6
matrix = np.random.rand(n_total, n_total) * 0.5 + 0.2
matrix = (matrix + matrix.T) / 2  # Make symmetric
np.fill_diagonal(matrix, 1.0)

# Add higher values for between-brain block to highlight it
matrix[0:3, 3:6] = np.random.rand(3, 3) * 0.3 + 0.6
matrix[3:6, 0:3] = matrix[0:3, 3:6].T

# Plot heatmap
im = ax3.imshow(matrix, cmap='RdYlBu_r', vmin=0, vmax=1, aspect='equal')
ax3.set_xticks(range(6))
ax3.set_yticks(range(6))
ax3.set_xticklabels(all_channels, fontsize=8, rotation=45, ha='right')
ax3.set_yticklabels(all_channels, fontsize=8)

# Add block boundaries
ax3.axhline(2.5, color='white', linewidth=3)
ax3.axvline(2.5, color='white', linewidth=3)

# Add block labels
ax3.text(1, -1.2, "P1", ha='center', fontsize=10, color=COLORS["signal_1"], fontweight='bold')
ax3.text(4, -1.2, "P2", ha='center', fontsize=10, color=COLORS["signal_2"], fontweight='bold')
ax3.text(-1.5, 1, "P1", ha='center', va='center', fontsize=10, color=COLORS["signal_1"], 
         fontweight='bold', rotation=90)
ax3.text(-1.5, 4, "P2", ha='center', va='center', fontsize=10, color=COLORS["signal_2"], 
         fontweight='bold', rotation=90)

# Colorbar
cbar = plt.colorbar(im, ax=ax3, shrink=0.8)
cbar.set_label("Connectivity", fontsize=9)

plt.tight_layout()
plt.show()

print("✓ Visualization 6: Hyperscanning data structure")
print("  A: Two data arrays combined into one")
print("  B: Connectivity matrix has 4 blocks (within-P1, within-P2, between, between.T)")
print("  C: Example 6×6 matrix with channel naming convention")

In [ ]:
# =============================================================================
# Utility Functions: Hyperscanning Data Structure
# =============================================================================

def create_hyperscanning_data_structure(
    data_p1: NDArray[np.float64],
    data_p2: NDArray[np.float64],
    channel_names_p1: List[str],
    channel_names_p2: List[str]
) -> Dict[str, Any]:
    """
    Create a unified data structure for hyperscanning analysis.
    
    Combines data from two participants into a single structure with
    proper channel labeling and metadata.
    
    Parameters
    ----------
    data_p1 : NDArray[np.float64]
        EEG data from participant 1, shape (n_channels_p1, n_samples).
    data_p2 : NDArray[np.float64]
        EEG data from participant 2, shape (n_channels_p2, n_samples).
    channel_names_p1 : List[str]
        Channel names for participant 1 (without prefix).
    channel_names_p2 : List[str]
        Channel names for participant 2 (without prefix).
    
    Returns
    -------
    Dict[str, Any]
        Dictionary containing:
        - 'data_combined': Combined data array (n_ch_total, n_samples)
        - 'channel_names': Channel names with P1_/P2_ prefixes
        - 'n_channels_p1': Number of channels for P1
        - 'n_channels_p2': Number of channels for P2
        - 'participant_labels': Array of 1s and 2s indicating participant
    
    Examples
    --------
    >>> data_p1 = np.random.randn(4, 1000)
    >>> data_p2 = np.random.randn(4, 1000)
    >>> names = ['Fz', 'Cz', 'Pz', 'Oz']
    >>> result = create_hyperscanning_data_structure(data_p1, data_p2, names, names)
    >>> result['data_combined'].shape
    (8, 1000)
    """
    # Validate inputs
    if data_p1.shape[1] != data_p2.shape[1]:
        raise ValueError("Both participants must have same number of samples")
    if len(channel_names_p1) != data_p1.shape[0]:
        raise ValueError("channel_names_p1 length must match data_p1 channels")
    if len(channel_names_p2) != data_p2.shape[0]:
        raise ValueError("channel_names_p2 length must match data_p2 channels")
    
    n_ch_p1 = data_p1.shape[0]
    n_ch_p2 = data_p2.shape[0]
    
    # Add prefixes to channel names
    names_p1_prefixed = [f"P1_{name}" for name in channel_names_p1]
    names_p2_prefixed = [f"P2_{name}" for name in channel_names_p2]
    
    # Combine data
    data_combined = np.vstack([data_p1, data_p2])
    channel_names_combined = names_p1_prefixed + names_p2_prefixed
    
    # Create participant labels
    participant_labels = np.array([1] * n_ch_p1 + [2] * n_ch_p2)
    
    return {
        "data_combined": data_combined,
        "channel_names": channel_names_combined,
        "n_channels_p1": n_ch_p1,
        "n_channels_p2": n_ch_p2,
        "participant_labels": participant_labels
    }


def extract_connectivity_blocks(
    full_matrix: NDArray[np.float64],
    n_channels_p1: int
) -> Dict[str, NDArray[np.float64]]:
    """
    Extract connectivity blocks from a combined hyperscanning matrix.
    
    Separates the full (n_total × n_total) matrix into within-participant
    and between-participant blocks.
    
    Parameters
    ----------
    full_matrix : NDArray[np.float64]
        Full connectivity matrix, shape (n_total, n_total).
    n_channels_p1 : int
        Number of channels belonging to participant 1.
    
    Returns
    -------
    Dict[str, NDArray[np.float64]]
        Dictionary containing:
        - 'within_p1': Connectivity within P1 (n_p1, n_p1)
        - 'within_p2': Connectivity within P2 (n_p2, n_p2)
        - 'between': Inter-brain connectivity (n_p1, n_p2)
    
    Examples
    --------
    >>> matrix = np.random.rand(8, 8)
    >>> blocks = extract_connectivity_blocks(matrix, n_channels_p1=4)
    >>> blocks['between'].shape
    (4, 4)
    """
    n_total = full_matrix.shape[0]
    n_p1 = n_channels_p1
    n_p2 = n_total - n_p1
    
    within_p1 = full_matrix[:n_p1, :n_p1]
    within_p2 = full_matrix[n_p1:, n_p1:]
    between = full_matrix[:n_p1, n_p1:]
    
    return {
        "within_p1": within_p1,
        "within_p2": within_p2,
        "between": between
    }


def combine_connectivity_blocks(
    within_p1: NDArray[np.float64],
    within_p2: NDArray[np.float64],
    between: NDArray[np.float64]
) -> NDArray[np.float64]:
    """
    Combine connectivity blocks into a full hyperscanning matrix.
    
    Assembles within-participant and between-participant blocks into
    the complete (n_p1 + n_p2) × (n_p1 + n_p2) matrix.
    
    Parameters
    ----------
    within_p1 : NDArray[np.float64]
        Connectivity within participant 1, shape (n_p1, n_p1).
    within_p2 : NDArray[np.float64]
        Connectivity within participant 2, shape (n_p2, n_p2).
    between : NDArray[np.float64]
        Inter-brain connectivity, shape (n_p1, n_p2).
    
    Returns
    -------
    NDArray[np.float64]
        Full connectivity matrix, shape (n_p1 + n_p2, n_p1 + n_p2).
    
    Examples
    --------
    >>> w1 = np.eye(3)
    >>> w2 = np.eye(3)
    >>> b = np.ones((3, 3)) * 0.5
    >>> full = combine_connectivity_blocks(w1, w2, b)
    >>> full.shape
    (6, 6)
    """
    n_p1 = within_p1.shape[0]
    n_p2 = within_p2.shape[0]
    n_total = n_p1 + n_p2
    
    full_matrix = np.zeros((n_total, n_total))
    
    # Fill in blocks
    full_matrix[:n_p1, :n_p1] = within_p1
    full_matrix[n_p1:, n_p1:] = within_p2
    full_matrix[:n_p1, n_p1:] = between
    full_matrix[n_p1:, :n_p1] = between.T
    
    return full_matrix


# Demo: Test the functions
print("Testing hyperscanning data structure functions...")

# Create simulated data
np.random.seed(42)
fs_demo = 256
duration_demo = 2
n_samples_demo = fs_demo * duration_demo

# Simulated EEG (4 channels each)
data_p1_demo = np.random.randn(4, n_samples_demo)
data_p2_demo = np.random.randn(4, n_samples_demo)
channels_demo = ["Fz", "Cz", "Pz", "Oz"]

# Create structure
hyper_data = create_hyperscanning_data_structure(
    data_p1_demo, data_p2_demo, channels_demo, channels_demo
)

print(f"\n✓ Combined data shape: {hyper_data['data_combined'].shape}")
print(f"✓ Channel names: {hyper_data['channel_names']}")
print(f"✓ Participant labels: {hyper_data['participant_labels']}")

# Create a dummy connectivity matrix
dummy_matrix = np.random.rand(8, 8)
dummy_matrix = (dummy_matrix + dummy_matrix.T) / 2  # Symmetric

# Extract blocks
blocks = extract_connectivity_blocks(dummy_matrix, n_channels_p1=4)
print(f"\n✓ Within-P1 shape: {blocks['within_p1'].shape}")
print(f"✓ Within-P2 shape: {blocks['within_p2'].shape}")
print(f"✓ Between shape: {blocks['between'].shape}")

# Reconstruct and verify
reconstructed = combine_connectivity_blocks(
    blocks['within_p1'], blocks['within_p2'], blocks['between']
)
is_equal = np.allclose(reconstructed, dummy_matrix)
print(f"\n✓ Reconstruction matches original: {is_equal}")

---
## Section 7: Preprocessing for Hyperscanning

All standard EEG preprocessing applies to hyperscanning (filtering, artifact rejection, re-referencing). However, hyperscanning introduces **specific considerations** that don't exist in single-brain studies.

### Critical: Time Synchronization

**This is THE most important preprocessing step for hyperscanning.**

Both recordings must be temporally aligned to the same time base. Without this, all inter-brain connectivity measures are meaningless.

Common synchronization methods:
- **Shared trigger signals**: Both systems receive the same TTL pulses
- **Photodiode markers**: Both record from shared visual stimulus
- **Hardware synchronization**: Systems share a clock

Always **verify** synchronization before analysis!

### Reference Scheme

Ideally, use the **same reference scheme** for both participants:
- **Average reference**: Most common in hyperscanning
- **Linked mastoids**: Also acceptable
- **Different references**: Can bias between-brain metrics

### Joint Artifact Rejection

In single-brain studies, you reject epochs with artifacts. In hyperscanning:

> **Reject epochs where EITHER participant has artifacts.**

Why? If P1 has clean data but P2 has an eye blink, comparing that epoch is like comparing apples to noise. You may lose more data than in single-subject studies, but the remaining data is valid for between-brain analysis.

### Movement Artifacts

Social interaction involves movement — gestures, speech, facial expressions. This creates:
- **EMG artifacts**: Especially in face/jaw muscles
- **Electrode movement**: If participants move their heads
- **Synchronized artifacts**: If movements are coordinated!

Solutions:
- Higher high-pass filter (e.g., 1-2 Hz instead of 0.1 Hz)
- ICA to remove muscle components
- Motion tracking (if available) as covariate

In [ ]:
# =============================================================================
# Visualization 7: Preprocessing Pipeline for Hyperscanning
# =============================================================================

fig, ax = plt.subplots(figsize=(14, 8))

# Pipeline stages as boxes
def draw_box(ax: plt.Axes, x: float, y: float, width: float, height: float,
             text: str, color: str, fontsize: int = 10) -> None:
    """Draw a labeled box."""
    box = Rectangle((x - width/2, y - height/2), width, height,
                    color=color, alpha=0.7, zorder=2)
    ax.add_patch(box)
    ax.text(x, y, text, ha='center', va='center', fontsize=fontsize,
            fontweight='bold', color='white' if color != COLORS["grid"] else COLORS["text"],
            zorder=3, wrap=True)

def draw_arrow(ax: plt.Axes, x1: float, y1: float, x2: float, y2: float,
               color: str = None) -> None:
    """Draw an arrow."""
    if color is None:
        color = COLORS["text"]
    ax.annotate("", xy=(x2, y2), xytext=(x1, y1),
               arrowprops=dict(arrowstyle="->", color=color, lw=2))

# Layout parameters
box_w = 0.22
box_h = 0.12
y_p1 = 0.75
y_p2 = 0.45
y_combined = 0.15

# Title
ax.text(0.5, 0.95, "Hyperscanning Preprocessing Pipeline", ha='center',
        fontsize=14, fontweight='bold', color=COLORS["text"])

# Stage 1: Raw data (two parallel streams)
ax.text(0.1, 0.9, "Participant 1", ha='center', fontsize=11, 
        fontweight='bold', color=COLORS["signal_1"])
draw_box(ax, 0.1, y_p1, box_w, box_h, "Raw EEG\n(P1)", COLORS["signal_1"])

ax.text(0.1, y_p2 + 0.17, "Participant 2", ha='center', fontsize=11,
        fontweight='bold', color=COLORS["signal_2"])
draw_box(ax, 0.1, y_p2, box_w, box_h, "Raw EEG\n(P2)", COLORS["signal_2"])

# Stage 2: Filtering (parallel)
draw_arrow(ax, 0.22, y_p1, 0.28, y_p1, COLORS["signal_1"])
draw_box(ax, 0.38, y_p1, box_w, box_h, "Filter\n(1-40 Hz)", COLORS["signal_1"])

draw_arrow(ax, 0.22, y_p2, 0.28, y_p2, COLORS["signal_2"])
draw_box(ax, 0.38, y_p2, box_w, box_h, "Filter\n(1-40 Hz)", COLORS["signal_2"])

# Stage 3: Artifact detection (parallel)
draw_arrow(ax, 0.50, y_p1, 0.56, y_p1, COLORS["signal_1"])
draw_box(ax, 0.66, y_p1, box_w, box_h, "Detect\nArtifacts", COLORS["signal_1"])

draw_arrow(ax, 0.50, y_p2, 0.56, y_p2, COLORS["signal_2"])
draw_box(ax, 0.66, y_p2, box_w, box_h, "Detect\nArtifacts", COLORS["signal_2"])

# Convergence: Synchronization check
ax.text(0.88, 0.9, "CRITICAL", ha='center', fontsize=10, 
        fontweight='bold', color=COLORS["negative"])
draw_arrow(ax, 0.78, y_p1, 0.82, (y_p1 + y_p2)/2 + 0.05, COLORS["high_sync"])
draw_arrow(ax, 0.78, y_p2, 0.82, (y_p1 + y_p2)/2 - 0.05, COLORS["high_sync"])
draw_box(ax, 0.88, (y_p1 + y_p2)/2, box_w, box_h * 1.3, "Verify\nSynchronization", 
         COLORS["high_sync"])

# Convergence arrow down
draw_arrow(ax, 0.88, (y_p1 + y_p2)/2 - 0.08, 0.66, y_combined + 0.08, COLORS["high_sync"])

# Joint artifact rejection
draw_box(ax, 0.66, y_combined, box_w * 1.2, box_h, "Joint Artifact\nRejection", COLORS["negative"])
ax.text(0.66, y_combined - 0.1, "Reject if EITHER\nparticipant has artifact",
        ha='center', fontsize=8, style='italic', color=COLORS["text"])

# Re-reference
draw_arrow(ax, 0.54, y_combined, 0.46, y_combined, COLORS["text"])
draw_box(ax, 0.38, y_combined, box_w, box_h, "Re-reference\n(Average)", COLORS["grid"])

# Clean data output
draw_arrow(ax, 0.26, y_combined, 0.18, y_combined, COLORS["text"])
draw_box(ax, 0.1, y_combined, box_w, box_h, "Clean\nSynchronized\nData", COLORS["positive"])

# Legend for critical steps
ax.text(0.03, 0.05, "Key: ", fontsize=9, fontweight='bold', color=COLORS["text"])
critical_marker = Rectangle((0.08, 0.04), 0.04, 0.025, color=COLORS["high_sync"])
ax.add_patch(critical_marker)
ax.text(0.13, 0.05, "Hyperscanning-specific step", fontsize=8, va='center', color=COLORS["text"])

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.axis('off')

plt.tight_layout()
plt.show()

print("✓ Visualization 7: Hyperscanning preprocessing pipeline")
print("  Two parallel streams converge at synchronization check")
print("  Joint artifact rejection: reject if EITHER participant has artifact")

---
## Section 8: Choosing Connectivity Metrics

We'll cover many connectivity metrics in the upcoming notebooks. How do you choose the right one for your research question?

### Decision Framework

| Research Question | Metric Type | Examples |
|-------------------|-------------|----------|
| Are phases aligned between brains? | Phase-based | PLV, PLI, wPLI |
| Do power fluctuations co-vary? | Amplitude-based | CCorr, PowCorr |
| Linear coupling at specific frequencies? | Coherence-based | COH, ImCoh |
| Any statistical dependency? | Information-theoretic | MI |
| Who influences whom (direction)? | Directed | Transfer Entropy, Granger |

### For Hyperscanning Specifically

**Phase Locking Value (PLV)**: Most commonly used in hyperscanning. Intuitive interpretation: "how often are the phases aligned?" Easy to compute and understand. *Caveat*: sensitive to volume conduction, but this is less of an issue between brains.

**Coherence (COH)**: Classic measure combining phase and amplitude. Well-established in the field. Good for exploratory analyses.

**Imaginary Coherence (ImCoh)**: Zero-lag robust. Even though volume conduction isn't an issue between brains, it's good practice and handles other zero-lag confounds.

**Phase Lag Index (PLI/wPLI)**: Most robust to artifacts. Conservative choice. May miss genuine zero-lag synchrony.

**Amplitude Correlation (CCorr)**: Captures a different mechanism than phase. Complementary to PLV. Less studied but equally valid.

**Transfer Entropy (TE)**: For directional questions: "Does brain A lead brain B?" More computationally intensive but answers unique questions.

### Frequency Band Considerations

Different frequency bands may show different effects:
- **Theta (4-8 Hz)**: Often linked to memory, coordination
- **Alpha (8-13 Hz)**: Attention, inhibition, widespread effects
- **Beta (13-30 Hz)**: Motor, social cognition
- **Gamma (30+ Hz)**: Local processing, harder to measure

**Recommendation**: Either have a *hypothesis* for a specific band, or report multiple bands (with appropriate multiple comparisons correction).

In [ ]:
# =============================================================================
# Visualization 8: Metric Selection Decision Tree (Simple Flowchart Style)
# =============================================================================

fig, ax = plt.subplots(figsize=(14, 10))

# Helper functions
def draw_decision(ax: plt.Axes, x: float, y: float, text: str, color: str,
                  width: float = 0.14, height: float = 0.08) -> None:
    """Draw a diamond-shaped decision node."""
    hw, hh = width/2, height/2
    diamond = plt.Polygon([(x, y + hh), (x + hw, y), (x, y - hh), (x - hw, y)],
                         color=color, alpha=0.4, edgecolor=color, linewidth=2, zorder=2)
    ax.add_patch(diamond)
    ax.text(x, y, text, ha='center', va='center', fontsize=9, 
            fontweight='bold', color=color, zorder=3)

def draw_box(ax: plt.Axes, x: float, y: float, text: str, color: str,
             width: float = 0.11, height: float = 0.055) -> None:
    """Draw a rectangular metric box."""
    box = Rectangle((x - width/2, y - height/2), width, height, 
                    color=color, alpha=0.8, zorder=2)
    ax.add_patch(box)
    ax.text(x, y, text, ha='center', va='center', fontsize=9, 
            fontweight='bold', color='white', zorder=3)

# Orthogonal connector: vertical down, then horizontal, then vertical down
def connect_down_branch(ax: plt.Axes, x1: float, y1: float, x2: float, y2: float,
                        label: str = None, label_pos: str = "left") -> None:
    """Draw L-shaped connector: down from x1, then horizontal to x2, then down to y2."""
    mid_y = (y1 + y2) / 2
    # Vertical segment from start
    ax.plot([x1, x1], [y1, mid_y], color=COLORS["text"], lw=1.5, zorder=1)
    # Horizontal segment
    ax.plot([x1, x2], [mid_y, mid_y], color=COLORS["text"], lw=1.5, zorder=1)
    # Vertical segment to end with arrow
    ax.annotate("", xy=(x2, y2), xytext=(x2, mid_y),
               arrowprops=dict(arrowstyle="->", color=COLORS["text"], lw=1.5))
    # Label
    if label:
        offset = -0.02 if label_pos == "left" else 0.02
        ha = "right" if label_pos == "left" else "left"
        ax.text(x1 + offset, mid_y + 0.02, label, fontsize=9, ha=ha, 
                color=COLORS["text"], fontweight='bold')

# Simple vertical connector
def connect_down(ax: plt.Axes, x: float, y1: float, y2: float, label: str = None) -> None:
    """Draw simple vertical arrow."""
    ax.annotate("", xy=(x, y2), xytext=(x, y1),
               arrowprops=dict(arrowstyle="->", color=COLORS["text"], lw=1.5))
    if label:
        ax.text(x + 0.02, (y1 + y2)/2, label, fontsize=9, ha='left', 
                color=COLORS["text"], fontweight='bold')

# Title
ax.text(0.5, 0.97, "Metric Selection Decision Tree", ha='center',
        fontsize=14, fontweight='bold', color=COLORS["text"])

# Layout - Y levels
L1 = 0.85  # Direction?
L2 = 0.68  # Directed metrics / Phase or Amplitude?
L3 = 0.51  # Amplitude metrics / Robustness?
L4 = 0.34  # Robust metrics / Include amplitude?
L5 = 0.17  # PLV / Coherence

# ═══════════════════════════════════════════════════════════════════════════
# LEVEL 1: Direction matters?
# ═══════════════════════════════════════════════════════════════════════════
draw_decision(ax, 0.5, L1, "Direction\nmatters?", COLORS["signal_4"])

# ═══════════════════════════════════════════════════════════════════════════
# LEVEL 2: Yes -> Directed | No -> Phase/Amplitude?
# ═══════════════════════════════════════════════════════════════════════════

# YES branch (left)
connect_down_branch(ax, 0.5, L1 - 0.04, 0.20, L2 + 0.04, "Yes", "left")

draw_box(ax, 0.13, L2, "Transfer\nEntropy", COLORS["signal_4"])
draw_box(ax, 0.27, L2, "Granger\nCausality", COLORS["signal_4"])
ax.text(0.20, L2 - 0.045, "Directed", ha='center', fontsize=9, 
        style='italic', color=COLORS["signal_4"])

# NO branch (right)
connect_down_branch(ax, 0.5, L1 - 0.04, 0.70, L2 + 0.04, "No", "right")

draw_decision(ax, 0.70, L2, "Phase or\nAmplitude?", COLORS["signal_3"])

# ═══════════════════════════════════════════════════════════════════════════
# LEVEL 3: Amplitude -> metrics | Phase -> Robustness?
# ═══════════════════════════════════════════════════════════════════════════

# AMPLITUDE branch (right-right)
connect_down_branch(ax, 0.70, L2 - 0.04, 0.88, L3 + 0.035, "Amplitude", "right")

draw_box(ax, 0.82, L3, "CCorr", COLORS["signal_5"])
draw_box(ax, 0.95, L3, "PowCorr", COLORS["signal_5"])
ax.text(0.885, L3 - 0.045, "Amplitude-based", ha='center', fontsize=9,
        style='italic', color=COLORS["signal_5"])

# PHASE branch (down-left from Phase/Amplitude)
connect_down_branch(ax, 0.70, L2 - 0.04, 0.50, L3 + 0.04, "Phase", "left")

draw_decision(ax, 0.50, L3, "Robustness\nneeded?", COLORS["signal_1"])

# ═══════════════════════════════════════════════════════════════════════════
# LEVEL 4: High -> robust metrics | Standard -> Include amplitude?
# ═══════════════════════════════════════════════════════════════════════════

# HIGH robustness (left)
connect_down_branch(ax, 0.50, L3 - 0.04, 0.25, L4 + 0.035, "High", "left")

draw_box(ax, 0.18, L4, "wPLI", COLORS["signal_6"])
draw_box(ax, 0.32, L4, "PLI", COLORS["signal_6"])
ax.text(0.25, L4 - 0.045, "Robust to artifacts", ha='center', fontsize=9,
        style='italic', color=COLORS["signal_6"])

# STANDARD (right)
connect_down_branch(ax, 0.50, L3 - 0.04, 0.70, L4 + 0.04, "Standard", "right")

draw_decision(ax, 0.70, L4, "Include\namplitude?", COLORS["signal_2"])

# ═══════════════════════════════════════════════════════════════════════════
# LEVEL 5: No -> PLV | Yes -> Coherence
# ═══════════════════════════════════════════════════════════════════════════

# NO -> PLV (left)
connect_down_branch(ax, 0.70, L4 - 0.04, 0.55, L5 + 0.035, "No", "left")

draw_box(ax, 0.55, L5, "PLV", COLORS["signal_1"])
ax.text(0.55, L5 - 0.045, "Most common", ha='center', fontsize=8,
        style='italic', color=COLORS["signal_1"])

# YES -> Coherence (right)
connect_down_branch(ax, 0.70, L4 - 0.04, 0.85, L5 + 0.035, "Yes", "right")

draw_box(ax, 0.78, L5, "COH", COLORS["signal_2"])
draw_box(ax, 0.92, L5, "ImCoh", COLORS["signal_2"])
ax.text(0.85, L5 - 0.045, "Coherence-based", ha='center', fontsize=8,
        style='italic', color=COLORS["signal_2"])

ax.set_xlim(0, 1)
ax.set_ylim(0.05, 1)
ax.axis('off')

plt.tight_layout()
plt.show()

print("✓ Visualization 8: Metric selection decision tree")
print("  Orthogonal flowchart style - cleaner and easier to follow")
print("  Each decision leads to appropriate metric choices")

---
## Section 9: Pseudo-Pair Analysis — The Key Control

This is perhaps the most important methodological concept in hyperscanning. If you remember only one thing from this notebook, let it be this.

### The Fundamental Question

When we observe inter-brain synchrony, we must ask:

> **Is this synchrony due to the INTERACTION, or would it occur anyway?**

### What Are Pseudo-Pairs?

A **pseudo-pair** is created by pairing participants who **never actually interacted**:
- Take P1 from real pair A
- Take P2 from real pair B
- Compute "synchrony" between them

These pseudo-pairs experienced the **same experimental conditions** (same task, same stimuli, same environment) but had **no social interaction** with each other.

### The Logic

If synchrony is driven by **shared stimuli** (both see the same screen, hear the same sounds), then pseudo-pairs should show similar synchrony to real pairs — both are processing the same input.

If synchrony is driven by **actual interaction** (conversation, coordination, rapport), then real pairs should show **higher synchrony** than pseudo-pairs — only real pairs had the interactive component.

### Statistical Test

1. Compute synchrony for all **real pairs** → distribution of real synchrony values
2. Compute synchrony for all **pseudo-pairs** → null distribution
3. Test: Is real synchrony significantly greater than pseudo-pair synchrony?

If **real > pseudo**: ✅ Synchrony is interaction-specific!
If **real ≈ pseudo**: ⚠️ Synchrony may just be shared stimulus response.
If **real < pseudo**: 🤔 Interesting! Interaction might actually *reduce* synchrony.

### Implementation Notes

- With $N$ participants in $N/2$ real pairs, you have many possible pseudo-pairs
- Can use all pseudo-pairs or subsample
- Test at the group level (are real pairs generally higher than pseudo?)
- Can also test individual pairs against the pseudo-pair null distribution

In [ ]:
# =============================================================================
# Visualization 9: Pseudo-Pair Concept
# =============================================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Left panel: Real pairs
ax1 = axes[0]
ax1.set_title("Real Pairs (Interacted)", fontsize=13, fontweight='bold', 
              color=COLORS["positive"], pad=15)

# Draw 3 real pairs
pair_colors = [COLORS["signal_1"], COLORS["signal_3"], COLORS["signal_5"]]
y_positions = [0.75, 0.45, 0.15]

for i, (y, color) in enumerate(zip(y_positions, pair_colors)):
    # Left person
    head_l = Circle((0.25, y), 0.08, color=color, alpha=0.7)
    ax1.add_patch(head_l)
    ax1.text(0.25, y - 0.13, f"P{2*i+1}", ha='center', fontsize=10, 
             fontweight='bold', color=color)
    
    # Right person
    head_r = Circle((0.75, y), 0.08, color=color, alpha=0.7)
    ax1.add_patch(head_r)
    ax1.text(0.75, y - 0.13, f"P{2*i+2}", ha='center', fontsize=10, 
             fontweight='bold', color=color)
    
    # Interaction arrow (double-headed, solid)
    ax1.annotate("", xy=(0.65, y), xytext=(0.35, y),
                arrowprops=dict(arrowstyle="<->", color=COLORS["high_sync"], lw=2))
    
    # Pair label
    ax1.text(0.5, y + 0.12, f"Pair {chr(65+i)}", ha='center', fontsize=9, 
             style='italic', color=COLORS["text"])

# Legend
ax1.text(0.5, -0.05, "↔ = Actual interaction during experiment", ha='center',
        fontsize=10, color=COLORS["high_sync"], fontweight='bold')

ax1.set_xlim(0, 1)
ax1.set_ylim(-0.1, 1)
ax1.axis('off')

# Right panel: Pseudo-pairs
ax2 = axes[1]
ax2.set_title("Pseudo-Pairs (Never Interacted)", fontsize=13, fontweight='bold',
              color=COLORS["negative"], pad=15)

# Draw pseudo-pairs: mix participants from different real pairs
pseudo_pairs = [(0, 3), (1, 4), (2, 5)]  # P1 with P4, P2 with P5, P3 with P6
pseudo_y = [0.75, 0.45, 0.15]
left_colors = [COLORS["signal_1"], COLORS["signal_1"], COLORS["signal_3"]]
right_colors = [COLORS["signal_3"], COLORS["signal_5"], COLORS["signal_5"]]

for i, (y, l_color, r_color) in enumerate(zip(pseudo_y, left_colors, right_colors)):
    p_left, p_right = pseudo_pairs[i]
    
    # Left person
    head_l = Circle((0.25, y), 0.08, color=l_color, alpha=0.7)
    ax2.add_patch(head_l)
    ax2.text(0.25, y - 0.13, f"P{p_left+1}", ha='center', fontsize=10, 
             fontweight='bold', color=l_color)
    
    # Right person
    head_r = Circle((0.75, y), 0.08, color=r_color, alpha=0.7)
    ax2.add_patch(head_r)
    ax2.text(0.75, y - 0.13, f"P{p_right+1}", ha='center', fontsize=10, 
             fontweight='bold', color=r_color)
    
    # NO interaction (dashed, crossed out)
    ax2.plot([0.35, 0.65], [y, y], color=COLORS["low_sync"], lw=2, ls='--')
    # Cross mark
    ax2.plot([0.48, 0.52], [y + 0.03, y - 0.03], color=COLORS["negative"], lw=2)
    ax2.plot([0.48, 0.52], [y - 0.03, y + 0.03], color=COLORS["negative"], lw=2)
    
    # Pseudo-pair label
    ax2.text(0.5, y + 0.12, f"Pseudo {i+1}", ha='center', fontsize=9,
             style='italic', color=COLORS["grid"])

# Legend
ax2.text(0.5, -0.05, "✗ = Same experiment, but never interacted", ha='center',
        fontsize=10, color=COLORS["negative"], fontweight='bold')

ax2.set_xlim(0, 1)
ax2.set_ylim(-0.1, 1)
ax2.axis('off')

plt.tight_layout()
plt.show()

print("✓ Visualization 9: Real pairs vs pseudo-pairs")
print("  Real pairs: participants who actually interacted during the experiment")
print("  Pseudo-pairs: participants from different real pairs (never interacted)")
print("  If synchrony is interaction-specific, real pairs > pseudo-pairs")

In [ ]:
# =============================================================================
# Visualization 10: Pseudo-Pair Null Distribution
# =============================================================================

np.random.seed(42)

# Simulate synchrony values
# Real pairs: higher synchrony (interaction effect)
real_pair_sync = np.random.normal(0.55, 0.08, 15)  # 15 real pairs
real_pair_sync = np.clip(real_pair_sync, 0, 1)

# Pseudo-pairs: lower synchrony (no interaction)
pseudo_pair_sync = np.random.normal(0.35, 0.10, 100)  # Many more pseudo-pairs
pseudo_pair_sync = np.clip(pseudo_pair_sync, 0, 1)

# Color scheme with good contrast:
# - Pseudo-pairs: Sky blue (neutral, clear)
# - Real pairs: Purple (high sync, stands out)
# - Conclusion text: Standard text color
pseudo_color = COLORS["signal_1"]  # Sky Blue - neutral null distribution
real_color = COLORS["high_sync"]   # Purple - real pairs (high sync)
success_color = COLORS["text"]     # Standard text color for conclusion

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left panel: Distributions
ax1 = axes[0]

# Pseudo-pair histogram (null distribution)
ax1.hist(pseudo_pair_sync, bins=20, alpha=0.7, color=pseudo_color,
         label="Pseudo-pairs (null)", density=True, edgecolor='white')

# Real pairs as vertical lines
for i, val in enumerate(real_pair_sync):
    ax1.axvline(val, color=real_color, alpha=0.8, linewidth=2.5,
               label="Real pairs" if i == 0 else None)

# Mean lines
ax1.axvline(pseudo_pair_sync.mean(), color=pseudo_color, linestyle='--', 
            linewidth=2.5, label=f"Pseudo mean: {pseudo_pair_sync.mean():.2f}")
ax1.axvline(real_pair_sync.mean(), color=real_color, linestyle='--',
            linewidth=2.5, label=f"Real mean: {real_pair_sync.mean():.2f}")

ax1.set_xlabel("Synchrony", fontsize=11)
ax1.set_ylabel("Density", fontsize=11)
ax1.set_title("Real Pairs vs Pseudo-Pair Null Distribution", fontsize=12, fontweight='bold')
ax1.legend(loc='upper right', fontsize=9)
ax1.set_xlim(0, 1)

# Right panel: Summary statistics
ax2 = axes[1]

# Bar plot comparing means
means = [pseudo_pair_sync.mean(), real_pair_sync.mean()]
stds = [pseudo_pair_sync.std(), real_pair_sync.std()]
colors_bars = [pseudo_color, real_color]
labels = ["Pseudo-pairs\n(null)", "Real pairs"]

bars = ax2.bar(labels, means, color=colors_bars, alpha=0.8, edgecolor='white', linewidth=2)
ax2.errorbar(labels, means, yerr=stds, fmt='none', color=COLORS["text"], 
             capsize=5, capthick=2, linewidth=2)

# Add significance annotation
from scipy import stats
t_stat, p_val = stats.ttest_ind(real_pair_sync, pseudo_pair_sync)

# Significance bracket
y_max = max(means) + max(stds) + 0.05
ax2.plot([0, 0, 1, 1], [y_max, y_max + 0.02, y_max + 0.02, y_max], 
         color=COLORS["text"], linewidth=1.5)

# Format p-value nicely
sig_text = "***" if p_val < 0.001 else ("**" if p_val < 0.01 else ("*" if p_val < 0.05 else "n.s."))
if p_val < 0.001:
    p_display = "p < 0.001"
else:
    p_display = f"p = {p_val:.3f}"

ax2.text(0.5, y_max + 0.04, f"{sig_text}\n{p_display}", ha='center', fontsize=10,
         color=real_color if p_val < 0.05 else COLORS["text"])

ax2.set_ylabel("Mean Synchrony", fontsize=11)
ax2.set_title("Statistical Comparison", fontsize=12, fontweight='bold')
ax2.set_ylim(0, 0.85)

# Add interpretation text
if real_pair_sync.mean() > pseudo_pair_sync.mean() and p_val < 0.05:
    conclusion = "✓ Synchrony is INTERACTION-SPECIFIC!"
    text_color = success_color
else:
    conclusion = "⚠ Synchrony may be stimulus-driven"
    text_color = COLORS["negative"]

ax2.text(0.5, 0.02, conclusion, ha='center', fontsize=12, fontweight='bold',
         color=text_color, transform=ax2.transAxes)

plt.tight_layout()
plt.show()

print("✓ Visualization 10: Pseudo-pair null distribution test")
print(f"  Real pairs mean: {real_pair_sync.mean():.3f} ± {real_pair_sync.std():.3f}")
print(f"  Pseudo-pairs mean: {pseudo_pair_sync.mean():.3f} ± {pseudo_pair_sync.std():.3f}")
print(f"  t-test p-value: {p_val:.2e}")
print(f"  Conclusion: Real pairs show significantly higher synchrony!")

---

## 10. Interpreting Inter-Brain Synchrony

**⏱️ Duration: 8 minutes**

### What Does Synchrony Actually Mean?

Finding inter-brain synchrony is exciting, but **interpretation requires caution**. Synchrony does NOT automatically mean:
- 🧠↔️🧠 Telepathic communication
- 💭 Understanding each other's thoughts
- ✅ Better interaction quality (always)

### Possible Mechanisms

**1. Shared Stimulus Processing**
Both participants perceive the same environment (sounds, visuals) → similar sensory processing → correlated brain activity. This is NOT truly "social" synchrony.
> **Control**: Pseudo-pair analysis

**2. Behavioral Coordination**
Synchronized movements (gestures, speech rhythm) → synchronized sensorimotor activity. May include movement artifacts, but also genuine coordination signatures.
> **Example**: Drummers synchronizing create motor cortex synchrony

**3. Predictive Coupling**
Brain A predicts Brain B's behavior and prepares responses; Brain B does the same. This interactive loop creates emergent synchrony.
> **Most "social"**: Reflects true interactive dynamics

**4. Common Physiological Rhythms**
Shared arousal, breathing synchronization, heart rate coupling. Less "cognitive" but still socially relevant.
> **Example**: Calm therapist → calm patient → physiological alignment

### What Synchrony Correlates With

| Finding | Domain | Reference |
|---------|--------|-----------|
| Task performance | Cooperation | Astolfi et al., 2010 |
| Subjective rapport | Conversation | Pérez et al., 2017 |
| Learning outcomes | Education | Dikker et al., 2017 |
| Therapeutic alliance | Clinical | Ramseyer & Tschacher, 2011 |
| Musical coordination | Performance | Lindenberger et al., 2009 |

### The Causality Question

```
Does synchrony CAUSE better interaction?
       ↕️ (or both?)
Does good interaction CAUSE synchrony?
```

**Current evidence is mostly correlational.** Experimental manipulations (disrupting synchrony, inducing it artificially) are active research frontiers.

> 💡 **Key insight**: Inter-brain synchrony is a **signature** of successful social interaction, but the causal mechanisms are still being investigated.

In [ ]:
# =============================================================================
# Visualization 11: Mechanisms of Inter-Brain Synchrony
# =============================================================================

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Helper to draw a simple head
def draw_head(ax: plt.Axes, x: float, y: float, radius: float, color: str, label: str) -> None:
    head = Circle((x, y), radius, facecolor=color, edgecolor=COLORS["text"], 
                  linewidth=2, alpha=0.7)
    ax.add_patch(head)
    ax.text(x, y, label, ha='center', va='center', fontsize=10, fontweight='bold',
            color=COLORS["text"])

# Panel 1: Shared Stimulus Processing
ax1 = axes[0, 0]
ax1.set_xlim(0, 10)
ax1.set_ylim(0, 8)
ax1.set_aspect('equal')
ax1.axis('off')
ax1.set_title("1. Shared Stimulus Processing", fontsize=12, fontweight='bold', pad=10)

# Screen in center
screen = Rectangle((4, 5), 2, 1.5, facecolor=COLORS["grid"], edgecolor=COLORS["text"], linewidth=2)
ax1.add_patch(screen)
ax1.text(5, 5.75, "SCREEN", fontsize=8, ha='center', va='center', fontweight='bold')

# Two participants looking at screen
draw_head(ax1, 2, 3, 0.8, COLORS["signal_1"], "P1")
draw_head(ax1, 8, 3, 0.8, COLORS["signal_2"], "P2")

# Arrows from screen to both
ax1.annotate('', xy=(2.5, 3.8), xytext=(4.2, 5), 
             arrowprops=dict(arrowstyle='->', color=COLORS["signal_4"], lw=2))
ax1.annotate('', xy=(7.5, 3.8), xytext=(5.8, 5),
             arrowprops=dict(arrowstyle='->', color=COLORS["signal_4"], lw=2))

# Similar waves under each head
t = np.linspace(0, 2*np.pi, 50)
ax1.plot(np.linspace(1, 3, 50), 1.5 + 0.3*np.sin(t), color=COLORS["signal_1"], lw=2)
ax1.plot(np.linspace(7, 9, 50), 1.5 + 0.3*np.sin(t), color=COLORS["signal_2"], lw=2)
ax1.text(5, 0.5, "Similar responses to same input\n(NOT interaction-specific)", 
         ha='center', fontsize=9, style='italic', color=COLORS["text"])

# Panel 2: Behavioral Coordination
ax2 = axes[0, 1]
ax2.set_xlim(0, 10)
ax2.set_ylim(0, 8)
ax2.set_aspect('equal')
ax2.axis('off')
ax2.set_title("2. Behavioral Coordination", fontsize=12, fontweight='bold', pad=10)

draw_head(ax2, 3, 4, 0.8, COLORS["signal_1"], "P1")
draw_head(ax2, 7, 4, 0.8, COLORS["signal_2"], "P2")

# Synchronized movement arrows
for y_off in [0.5, -0.5]:
    ax2.annotate('', xy=(4, 4 + y_off), xytext=(3.8, 4 + y_off),
                 arrowprops=dict(arrowstyle='->', color=COLORS["signal_1"], lw=2))
    ax2.annotate('', xy=(6, 4 + y_off), xytext=(6.2, 4 + y_off),
                 arrowprops=dict(arrowstyle='->', color=COLORS["signal_2"], lw=2))

# Synchronized waves
ax2.plot(np.linspace(1.5, 4.5, 50), 2 + 0.3*np.sin(t), color=COLORS["signal_1"], lw=2)
ax2.plot(np.linspace(5.5, 8.5, 50), 2 + 0.3*np.sin(t), color=COLORS["signal_2"], lw=2)

# Connection between movements
ax2.plot([4.2, 5.8], [4, 4], '--', color=COLORS["high_sync"], lw=2)
ax2.text(5, 6.5, "[ Joint Action ]", fontsize=11, ha='center', fontweight='bold',
         color=COLORS["high_sync"])
ax2.text(5, 0.5, "Synchronized actions -> synchronized motor activity", 
         ha='center', fontsize=9, style='italic', color=COLORS["text"])

# Panel 3: Predictive Coupling
ax3 = axes[1, 0]
ax3.set_xlim(0, 10)
ax3.set_ylim(0, 8)
ax3.set_aspect('equal')
ax3.axis('off')
ax3.set_title("3. Predictive Coupling", fontsize=12, fontweight='bold', pad=10)

draw_head(ax3, 3, 4, 0.8, COLORS["signal_1"], "P1")
draw_head(ax3, 7, 4, 0.8, COLORS["signal_2"], "P2")

# Bidirectional arrows with labels
ax3.annotate('', xy=(6, 4.5), xytext=(4, 4.5),
             arrowprops=dict(arrowstyle='->', color=COLORS["high_sync"], lw=2.5))
ax3.annotate('', xy=(4, 3.5), xytext=(6, 3.5),
             arrowprops=dict(arrowstyle='->', color=COLORS["high_sync"], lw=2.5))

ax3.text(5, 5.2, "Predict & respond", fontsize=9, ha='center', color=COLORS["high_sync"])
ax3.text(5, 2.8, "Adjust & adapt", fontsize=9, ha='center', color=COLORS["high_sync"])

# Question marks for prediction
ax3.text(2.2, 5.5, "?", fontsize=16, fontweight='bold', color=COLORS["signal_1"])
ax3.text(7.8, 5.5, "?", fontsize=16, fontweight='bold', color=COLORS["signal_2"])

ax3.text(5, 0.5, "Interactive loop creates emergent synchrony\n(Most 'social' mechanism)", 
         ha='center', fontsize=9, style='italic', color=COLORS["text"])

# Panel 4: Physiological Coupling
ax4 = axes[1, 1]
ax4.set_xlim(0, 10)
ax4.set_ylim(0, 8)
ax4.set_aspect('equal')
ax4.axis('off')
ax4.set_title("4. Physiological Coupling", fontsize=12, fontweight='bold', pad=10)

draw_head(ax4, 3, 4, 0.8, COLORS["signal_1"], "P1")
draw_head(ax4, 7, 4, 0.8, COLORS["signal_2"], "P2")

# Heart symbols as text
ax4.text(3, 2.5, "<3", fontsize=14, ha='center', color=COLORS["negative"], fontweight='bold')
ax4.text(7, 2.5, "<3", fontsize=14, ha='center', color=COLORS["negative"], fontweight='bold')

# Synchronized heartbeat waves
heartbeat_t = np.linspace(0, 4*np.pi, 100)
heartbeat = np.sin(heartbeat_t) * np.exp(-0.1 * np.abs(heartbeat_t % (2*np.pi) - np.pi))
ax4.plot(np.linspace(1.5, 4.5, 100), 1.5 + 0.3*heartbeat, color=COLORS["negative"], lw=2)
ax4.plot(np.linspace(5.5, 8.5, 100), 1.5 + 0.3*heartbeat, color=COLORS["negative"], lw=2)

# Connection
ax4.plot([4, 6], [2.5, 2.5], ':', color=COLORS["negative"], lw=2)

ax4.text(5, 6, "Breathing, arousal, heart rate", fontsize=10, ha='center')
ax4.text(5, 0.5, "Shared physiological state\n(Less cognitive, but socially meaningful)", 
         ha='center', fontsize=9, style='italic', color=COLORS["text"])

plt.tight_layout()
plt.show()

print("✓ Visualization 11: Four mechanisms of inter-brain synchrony")
print("  1. Shared stimulus -> similar sensory processing (control with pseudo-pairs)")
print("  2. Behavioral coordination -> synchronized motor/sensory activity")
print("  3. Predictive coupling -> interactive dynamics (most 'social')")
print("  4. Physiological coupling -> shared arousal state")

---

## 11. HyPyP and the Tool Ecosystem

**⏱️ Duration: 5 minutes**

### HyPyP: Hyperscanning Python Pipeline

**HyPyP** is an open-source Python library specifically designed for hyperscanning analysis:

| Feature | Description |
|---------|-------------|
| **Built on MNE-Python** | Leverages the powerful MNE ecosystem |
| **Connectivity metrics** | PLV, coherence, correlation, and more |
| **Statistical analysis** | Surrogates, permutation tests, pseudo-pairs |
| **Visualization** | Topoplots, circular plots, matrices |
| **Data structures** | Handles dual-subject data natively |

> 📚 **Reference**: Ayrolles et al. (2021). "HyPyP: A toolkit for hyperscanning analysis"

### Our Approach in This Workshop

```
┌───────────────────────────────────────────────────────────┐
│                       UNDERSTAND                          │
│  Build from scratch → know WHAT the metrics compute       │
├───────────────────────────────────────────────────────────┤
│                        COMPARE                            │
│  Validate against HyPyP → ensure correctness              │
├───────────────────────────────────────────────────────────┤
│                         APPLY                             │
│  Use HyPyP/MNE in practice → efficient production code    │
└───────────────────────────────────────────────────────────┘
```

### Tool Ecosystem Overview

| Tool | Purpose | Relationship |
|------|---------|--------------|
| **MNE-Python** | EEG/MEG analysis foundation | Core dependency |
| **mne-connectivity** | Single-brain connectivity | Base for metrics |
| **HyPyP** | Hyperscanning-specific | Our comparison target |
| **This workshop** | Educational implementations | Understanding-focused |
| **BCT (MATLAB)** | Graph theory metrics | Graph analysis inspiration |

### In Each Metric Notebook

For every connectivity metric (PLV, coherence, etc.), we will:

1. **Intuition**: What does this metric capture?
2. **Mathematics**: The actual formula
3. **Implementation**: Code from scratch
4. **Validation**: Compare to HyPyP
5. **Application**: Use on hyperscanning data
6. **Interpretation**: What do results mean?

> 💡 **Goal**: You'll be able to use HyPyP confidently AND know exactly what's happening under the hood!

---

## 12. Complete Hyperscanning Pipeline

**Duration: 5 minutes**

### End-to-End Workflow

A complete hyperscanning analysis follows these steps:

```
┌───────────────────────────────────────────────────────────┐
│  STEP 1: Data Loading & Synchronization                   │
│    - Load recordings from both participants               │
│    - Verify temporal alignment (trigger channels)         │
│    - Interpolate to common time base if needed            │
└───────────────────────────────────────────────────────────┘
                            ↓
┌───────────────────────────────────────────────────────────┐
│  STEP 2: Preprocessing                                    │
│    - Filter to frequency band of interest                 │
│    - Artifact rejection (BOTH participants must be clean) │
│    - Re-reference (same scheme for both)                  │
│    - Bad channel interpolation                            │
└───────────────────────────────────────────────────────────┘
                            ↓
┌───────────────────────────────────────────────────────────┐
│  STEP 3: Create Hyperscanning Data Structure              │
│    - Combine data with P1_, P2_ channel prefixes          │
│    - Create participant labels                            │
│    - Organize for connectivity analysis                   │
└───────────────────────────────────────────────────────────┘
                            ↓
┌───────────────────────────────────────────────────────────┐
│  STEP 4: Compute Connectivity                             │
│    - Within-P1 block (n_p1 × n_p1)                        │
│    - Within-P2 block (n_p2 × n_p2)                        │
│    - Between-brain block (n_p1 × n_p2)                    │
│    - Full combined matrix ((n_p1+n_p2) × (n_p1+n_p2))     │
└───────────────────────────────────────────────────────────┘
                            ↓
┌───────────────────────────────────────────────────────────┐
│  STEP 5: Statistical Testing                              │
│    - Surrogate distribution (phase shuffling)             │
│    - Pseudo-pair comparison                               │
│    - Multiple comparisons correction                      │
└───────────────────────────────────────────────────────────┘
                            ↓
┌───────────────────────────────────────────────────────────┐
│  STEP 6: Visualization & Interpretation                   │
│    - Connectivity matrices                                │
│    - Significant connections                              │
│    - Summary statistics                                   │
│    - Relate to behavioral/clinical outcomes               │
└───────────────────────────────────────────────────────────┘
```

### Key Code Functions (Preview)

We've defined utility functions earlier in this notebook:

| Function | Purpose |
|----------|---------|
| `create_hyperscanning_data_structure()` | Combine P1 + P2 data |
| `extract_connectivity_blocks()` | Split full matrix into blocks |
| `combine_connectivity_blocks()` | Merge blocks into full matrix |

In subsequent notebooks, we'll add connectivity computation functions (PLV, coherence, etc.) that integrate with this pipeline.

---

## 13. What's Coming Next

**Duration: 3 minutes**

### Your Journey Through Connectivity Metrics

Now that you understand the hyperscanning framework, you're ready to learn the specific metrics!

```
                    ┌─────────────────────────────────┐
                    │   E02: Introduction to          │
                    │   Hyperscanning (YOU ARE HERE)  │
                    └───────────────┬─────────────────┘
                                    │
          ┌─────────────────────────┼─────────────────────────┐
          │                         │                         │
          ▼                         ▼                         ▼
┌─────────────────┐      ┌─────────────────┐      ┌─────────────────┐
│  BLOCK F        │      │  BLOCK G        │      │  BLOCK H        │
│  Coherence      │      │  Phase-Based    │      │  Amplitude      │
├─────────────────┤      ├─────────────────┤      ├─────────────────┤
│ F01: Spectral   │      │ G01: PLV        │      │ H01: Envelope   │
│      Coherence  │      │ G02: PLI        │      │      Correlation│
│ F02: Imaginary  │      │ G03: wPLI       │      │ H02: Power      │
│      Coherence  │      │                 │      │      Correlation│
└─────────────────┘      └─────────────────┘      └─────────────────┘
```

### What Each Block Covers

| Block | Focus | Key Question |
|-------|-------|--------------|
| **F: Coherence** | Frequency-domain coupling | Are signals linearly related at each frequency? |
| **G: Phase** | Phase relationship | Are phases aligned across signals? |
| **H: Amplitude** | Power fluctuations | Do power envelopes co-vary? |

### In Each Metric Notebook

1. **Intuition**: What does this metric capture?
2. **Mathematics**: The formula, step by step
3. **Implementation**: Python code from scratch
4. **Visualization**: See what the metric reveals
5. **HyPyP Comparison**: Validate against production tools
6. **Hyperscanning Application**: Apply to inter-brain analysis
7. **Interpretation Guide**: What do results mean?

> **Ready to dive in?** Next up: **F01 - Spectral Coherence**

---

## 14. Summary

### Key Takeaways

| Concept | Key Point |
|---------|-----------|
| **Hyperscanning** | Simultaneous recording from 2+ interacting people |
| **Inter-brain synchrony** | Coupling between brain signals of different people |
| **No volume conduction** | Between-brain: no shared conduction (advantage!) |
| **Confounds** | Behavioral sync, shared stimuli, movement artifacts |
| **Pseudo-pairs** | Essential control for interaction specificity |
| **Data structure** | Within-P1, Within-P2, Between blocks |
| **Metric choice** | Depends on phase vs amplitude, directed vs undirected |

### What We Covered

1. **Definition**: Hyperscanning studies social brain IN social context
2. **Applications**: Education, clinical, development, performance
3. **Paradigms**: Cooperation, communication, joint attention, imitation
4. **Challenges**: Behavioral confounds, stimulus-driven effects
5. **Data organization**: Combined matrices with clear labeling
6. **Preprocessing**: Synchronization, joint artifact rejection
7. **Metric selection**: PLV, coherence, correlation, transfer entropy
8. **Pseudo-pairs**: The key control analysis
9. **Interpretation**: Correlation vs causation, multiple mechanisms

### Utility Functions Created

```python
# Data structure
create_hyperscanning_data_structure(data_p1, data_p2, channels_p1, channels_p2)
extract_connectivity_blocks(full_matrix, n_channels_p1)
combine_connectivity_blocks(within_p1, within_p2, between)
```

### Ready for Connectivity Metrics!

You now have the conceptual foundation to understand:
- **WHY** we measure inter-brain connectivity
- **WHAT** challenges we face
- **HOW** to structure and analyze hyperscanning data

Next: Learn the specific metrics (coherence, PLV, PLI, etc.) and apply them!

---

## 15. Discussion Questions

1. **Experimental Design**: You want to study parent-child attachment through hyperscanning. What paradigm would you design? What frequencies might be most relevant? What confounds would you worry about?

2. **Addressing Criticism**: A reviewer criticizes your study: "This synchrony is just because both participants see the same screen." How do you respond? What analyses would address this concern?

3. **Causality**: Inter-brain synchrony during therapy predicts treatment outcome. Does this mean we should try to INCREASE synchrony to improve treatment? What cautions would you raise?

4. **Frequency Specificity**: You find theta synchrony during cooperation but alpha synchrony during competition. What might explain frequency-specific effects in different social contexts?

5. **Remote Interaction**: Your hyperscanning study involves video calls (remote interaction). What additional challenges does this introduce compared to face-to-face interaction?

---

## 16. Exercises

### Exercise 1: Data Structure Practice
Using the simulated data from this notebook:
- Create a hyperscanning data structure with 6 channels per participant
- Verify that channel names are correctly prefixed
- Extract the between-brain block and compute its mean value

### Exercise 2: Pseudo-Pair Simulation
- Generate data for 6 "participants" forming 3 real pairs
- Create all possible pseudo-pairs
- Compute simple correlation for real vs pseudo pairs
- Is there a significant difference?

### Exercise 3: Metric Selection
For each scenario, choose the most appropriate metric and justify:
- a) "We want to know if phases align during conversation" 
- b) "We want to know who influences whom during teaching"
- c) "We need a robust measure for clinical application"

### Exercise 4: Critical Thinking
You find that inter-brain synchrony is LOWER for real pairs than pseudo-pairs during a competitive task. What might explain this counterintuitive finding?

---

*Notebook completed. Proceed to F01: Spectral Coherence.*

In [ ]:
# =============================================================================
# Exercise 1 Solution: Data Structure Practice
# =============================================================================

print("=" * 60)
print("Exercise 1: Data Structure Practice")
print("=" * 60)

# Create data for 6 channels per participant
channels_ex1_p1 = ["Fz", "Cz", "Pz", "F3", "F4", "Oz"]
channels_ex1_p2 = ["Fz", "Cz", "Pz", "F3", "F4", "Oz"]

fs_ex1 = 256
duration_ex1 = 5
n_samples_ex1 = fs_ex1 * duration_ex1

# Simulate data
np.random.seed(123)
data_ex1_p1 = np.random.randn(len(channels_ex1_p1), n_samples_ex1)
data_ex1_p2 = np.random.randn(len(channels_ex1_p2), n_samples_ex1)

# Create hyperscanning data structure
def create_hyperscanning_structure(
    data_p1: NDArray[np.floating],
    data_p2: NDArray[np.floating],
    channels_p1: List[str],
    channels_p2: List[str],
    fs: int
) -> Dict[str, Any]:
    """Create a hyperscanning data structure from two participants' data."""
    # Prefix channel names
    prefixed_p1 = [f"P1_{ch}" for ch in channels_p1]
    prefixed_p2 = [f"P2_{ch}" for ch in channels_p2]
    
    # Combine data
    combined_data = np.vstack([data_p1, data_p2])
    combined_channels = prefixed_p1 + prefixed_p2
    
    # Create labels
    labels = np.array([1] * len(channels_p1) + [2] * len(channels_p2))
    
    return {
        "data": combined_data,
        "channels": combined_channels,
        "participant_labels": labels,
        "n_channels_p1": len(channels_p1),
        "n_channels_p2": len(channels_p2),
        "fs": fs
    }

hyper_ex1 = create_hyperscanning_structure(
    data_ex1_p1, data_ex1_p2, channels_ex1_p1, channels_ex1_p2, fs_ex1
)

print(f"Combined channels: {hyper_ex1['channels']}")
print(f"Data shape: {hyper_ex1['data'].shape}")
print(f"Participant labels: {hyper_ex1['participant_labels']}")

# Extract between-brain block
n_p1 = hyper_ex1["n_channels_p1"]
n_p2 = hyper_ex1["n_channels_p2"]

# Compute simple correlation matrix
from scipy.stats import pearsonr
corr_matrix = np.zeros((n_p1 + n_p2, n_p1 + n_p2))
for i in range(n_p1 + n_p2):
    for j in range(n_p1 + n_p2):
        corr_matrix[i, j], _ = pearsonr(hyper_ex1["data"][i], hyper_ex1["data"][j])

# Extract between-brain block (P1 rows, P2 columns)
between_block = corr_matrix[:n_p1, n_p1:]
print(f"\nBetween-brain block shape: {between_block.shape}")
print(f"Between-brain block mean: {between_block.mean():.4f}")
print("(Expected close to 0 for random data)")

In [ ]:
# =============================================================================
# Exercise 2 Solution: Pseudo-Pair Simulation
# =============================================================================

print("=" * 60)
print("Exercise 2: Pseudo-Pair Simulation")
print("=" * 60)

np.random.seed(456)

# Generate 6 participants forming 3 real pairs
# Real pairs have correlated signals (shared component)
n_participants = 6
n_samples_ex2 = 1000

# Shared component for each real pair
shared_1 = np.random.randn(n_samples_ex2)
shared_2 = np.random.randn(n_samples_ex2)
shared_3 = np.random.randn(n_samples_ex2)

# Individual noise
noise_level = 0.5

participants = {
    "P1": 0.7 * shared_1 + noise_level * np.random.randn(n_samples_ex2),  # Pair 1
    "P2": 0.7 * shared_1 + noise_level * np.random.randn(n_samples_ex2),  # Pair 1
    "P3": 0.7 * shared_2 + noise_level * np.random.randn(n_samples_ex2),  # Pair 2
    "P4": 0.7 * shared_2 + noise_level * np.random.randn(n_samples_ex2),  # Pair 2
    "P5": 0.7 * shared_3 + noise_level * np.random.randn(n_samples_ex2),  # Pair 3
    "P6": 0.7 * shared_3 + noise_level * np.random.randn(n_samples_ex2),  # Pair 3
}

# Real pairs
real_pairs = [("P1", "P2"), ("P3", "P4"), ("P5", "P6")]

# All possible pseudo-pairs (participants from different real pairs)
pseudo_pairs_ex2 = [
    ("P1", "P3"), ("P1", "P4"), ("P1", "P5"), ("P1", "P6"),
    ("P2", "P3"), ("P2", "P4"), ("P2", "P5"), ("P2", "P6"),
    ("P3", "P5"), ("P3", "P6"),
    ("P4", "P5"), ("P4", "P6"),
]

# Compute correlations
real_correlations = []
for p1, p2 in real_pairs:
    corr, _ = pearsonr(participants[p1], participants[p2])
    real_correlations.append(corr)

pseudo_correlations = []
for p1, p2 in pseudo_pairs_ex2:
    corr, _ = pearsonr(participants[p1], participants[p2])
    pseudo_correlations.append(corr)

print(f"Real pairs: {real_pairs}")
print(f"Real pair correlations: {[f'{c:.3f}' for c in real_correlations]}")
print(f"Real mean: {np.mean(real_correlations):.3f}")
print(f"\nNumber of pseudo-pairs: {len(pseudo_pairs_ex2)}")
print(f"Pseudo-pair mean: {np.mean(pseudo_correlations):.3f}")

# Statistical test
t_ex2, p_ex2 = stats.ttest_ind(real_correlations, pseudo_correlations)
print(f"\nt-test: t = {t_ex2:.3f}, p = {p_ex2:.4f}")
print(f"Significant difference: {'YES' if p_ex2 < 0.05 else 'NO'}")

In [ ]:
# =============================================================================
# Exercise 3 Solution: Metric Selection
# =============================================================================

print("=" * 60)
print("Exercise 3: Metric Selection")
print("=" * 60)

solutions_ex3 = """
a) "We want to know if phases align during conversation"
   → PHASE-LOCKING VALUE (PLV)
   Justification: PLV measures phase consistency across trials/time.
   It captures whether neural oscillations are aligned in phase,
   which is the core question here.

b) "We want to know who influences whom during teaching"
   → TRANSFER ENTROPY or GRANGER CAUSALITY
   Justification: These are directional metrics that can reveal
   information flow from teacher to student (or vice versa).
   Transfer entropy is particularly suited for nonlinear relationships.

c) "We need a robust measure for clinical application"
   → WEIGHTED PHASE LAG INDEX (wPLI) or IMAGINARY COHERENCE
   Justification: These metrics are robust to volume conduction,
   which is critical for clinical reliability. wPLI also handles
   noise well and is less sensitive to outliers.
"""
print(solutions_ex3)

In [ ]:
# =============================================================================
# Exercise 4 Solution: Critical Thinking
# =============================================================================

print("=" * 60)
print("Exercise 4: Critical Thinking")
print("=" * 60)

solution_ex4 = """
Finding: Inter-brain synchrony LOWER for real pairs than pseudo-pairs
during competition.

Possible explanations:

1. ACTIVE DESYNCHRONIZATION
   During competition, participants may actively try to be unpredictable.
   Being synchronized would make you predictable to your opponent.
   → Lower synchrony = successful competitive strategy

2. DIFFERENT STRATEGIES
   Real competitors develop complementary (not similar) strategies.
   They occupy different "cognitive niches" to gain advantage.
   → Functional differentiation, not coordination

3. STIMULUS-DRIVEN BASELINE
   If the task has strong visual/auditory components, pseudo-pairs
   might show HIGH stimulus-locked synchrony simply from processing
   the same sensory input.
   Real pairs might show LOWER stimulus-locking because they're
   focused on opponent modeling rather than stimulus processing.

4. AROUSAL DIFFERENCES
   Competition increases arousal, which can desynchronize neural activity.
   Real competitive pairs have higher arousal than pseudo-pairs.

5. ATTENTION ALLOCATION
   Real competitors attend to different aspects (opponent's actions)
   while pseudo-pairs both attend to the same task elements.

Key insight: "Higher synchrony = better" is NOT always true!
The meaning of synchrony depends entirely on the context.
"""
print(solution_ex4)

print("\n" + "=" * 60)
print("✓ All exercises completed!")
print("=" * 60)